In [ ]:
from google.colab import drive

# Google Drive'ı bağlama
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import files
from google.colab import drive
import shutil
import os
# 2. Dosyayı yükle (seçmeli)
uploaded = files.upload()   # colabe dosyaa yükleme


Saving tekstil3test.zip to tekstil3test.zip


In [ ]:
#colabde yüklü dosyayı başka yere taşıma
from google.colab import files
from google.colab import drive
import shutil
import os
# 3. Her yüklenen dosyayı Drive'daki aynı adlı klasöre taşı
for filename in uploaded.keys():
    # Drive'daki hedef klasör yolu (örnek: MyDrive/tekstil3test)
    drive_folder = f"#your file path{filename.split('.')[0]}"  # klasör adı: dosya adı (uzantısız)

    # Eğer klasör yoksa oluştur
    if not os.path.exists(drive_folder):
        os.makedirs(drive_folder)

    # Yüklenen dosyanın geçici yolu
    src_path = f"/content/{filename}"

    # Drive'daki hedef dosya yolu
    dest_path = os.path.join(drive_folder, filename)

    # Dosyayı taşı
    shutil.move(src_path, dest_path)

    print(f"{filename} dosyası Drive'daki {drive_folder} klasörüne taşındı.")

In [ ]:
# colabde yüklü dosyayı bir klsörde açma zipli olan dosyayı
import zipfile
import os

drive_folder = "#your file path"  # açmak istediğin klasör yolu
zip_path = os.path.join(drive_folder, "tekstil3test.zip")  # zip dosyanın tam adı

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(drive_folder)

print("Zip dosyası başarıyla açıldı.")


Zip dosyası başarıyla açıldı.


In [ ]:
# faster r cnn ile eğtitirken oluşan bir json annotations dosyasınının json da categories ksımını değiştirme
import json
import os

# Dosya yolu (train annotations dosyası)
train_ann_path = "#your file path/train/_annotations.coco.json"

# Eğitimdeki sınıflar (supercategory olmadan)
categories_new = [
    {"id": 0, "name": "defect"},
    {"id": 1, "name": "1- patlak"},
    {"id": 2, "name": "2- igne_kirigi"},
    {"id": 3, "name": "3- jut"},
    {"id": 4, "name": "5- likra_kacigi"},
    {"id": 5, "name": "6- yag_lekesi"},
    {"id": 6, "name": "8- May cizgisi"},
    {"id": 7, "name": "fsa"}   # fsa default olarak eklenen class adı
]

# JSON dosyasını aç
with open(train_ann_path, "r") as f:
    data = json.load(f)

# Mevcut categories'den id:name map
old_cat_id_to_name = {cat["id"]: cat["name"] for cat in data["categories"]}

# Yeni categories name:id map (kolay eşleme için)
new_cat_name_to_id = {cat["name"]: cat["id"] for cat in categories_new}

# Annotationlardaki category_id'leri yeni id'lere eşle
for ann in data["annotations"]:
    old_id = ann["category_id"]
    old_name = old_cat_id_to_name.get(old_id)
    if old_name is None:
        print(f"Uyarı: Eski ID {old_id} için isim bulunamadı!")
        continue
    new_id = new_cat_name_to_id.get(old_name)
    if new_id is None:
        print(f"Uyarı: Yeni kategori listesinde '{old_name}' yok!")
        continue
    ann["category_id"] = new_id

# categories kısmını tamamen güncelle, supercategory olmadan
data["categories"] = categories_new

# Dosyayı kaydet (istersen farklı isimde)
fixed_train_ann_path = "#your file path/train/_annotations_fixed.coco.json"
with open(fixed_train_ann_path, "w") as f:
    json.dump(data, f, indent=4)

print(f"Train annotation dosyası güncellendi ve {fixed_train_ann_path} olarak kaydedildi.")


In [ ]:
import json

# Dosya yolu (örnek: train klasöründeki dosya)
json_path = "#your file path/train/ikinci.coco.json"

# Dosyayı oku
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Kategorileri yazdır
print("📦 Categories:")
for cat in data.get("categories", []):
    print(f"🆔 {cat['id']} - {cat['name']}")


In [ ]:
#FASTER R CNN İÇİN PAKET VE DOSYA KURULUMU


# Mevcut Colab ortamındaki PyTorch ve diğer paketlerin bağımlılık çakışmasını gidermek için:
# 1. Önce, Colab'in önceden yüklediği ve çakışmaya neden olabilecek tüm paketleri kaldırıyoruz.
!pip uninstall -y detectron2 fvcore torch torchvision torchaudio fastai
# 2. Ardından, Colab'deki mevcut CUDA sürümüyle uyumlu PyTorch sürümünü kuruyoruz.
!pip install torch torchvision torchaudio
# 3. Son olarak, Detectron2'yi doğrudan GitHub deposundan kuruyoruz.
#

In [ ]:
#FASTER R CNN İÇİN PAKET VE DOSYA KURULUMU
!pip install 'git+https://github.com/facebookresearch/detectron2.git'


In [ ]:
#YENİ ANNATATİONS OLUŞTURMA
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

import torch
import os
import json
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from PIL import Image

# --- Dosya Yolları ve Sınıf İsimleri ---
# Kendi dosya yollarınızı burada güncelleyin
model_path = "#your file path/model_final.pth"   # hazırlanan modelin drive kaydı
config_path = "#your file path/config.yaml"
data_dir = "/#your file path/train/"
test_annotations_path = os.path.join(data_dir, "_annotations_fixed.coco.json")
output_annotations_path = os.path.join(data_dir, "_annotations_fixed2.coco.json")

# Kendi sınıf listenizi buraya girin (YAML dosyasından da çekilebilir, ancak bu daha kolay)
class_names = [
    "defect",
    "1- patlak",
    "2- igne_kirigi",
    "3- jut",
    "5- likra_kacigi",
    "6- yag_lekesi",
    "8- May cizgisi",
    "fsa"
]

# --- 1. Detectron2 Dataset'i Kaydetme ---
try:
    register_coco_instances("my_dataset_test", {}, test_annotations_path, data_dir)
    MetadataCatalog.get("my_dataset_test").set(thing_classes=class_names)
    print("Dataset registered successfully.")
except Exception as e:
    print(f"Error registering dataset: {e}")

# --- 2. Modeli ve Konfigürasyonu Yükleme ---
cfg = get_cfg()
cfg.merge_from_file(config_path)

cfg.MODEL.WEIGHTS = model_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2  # Güvenilirlik eşiği (trash_score) 0.3
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("Model loaded with Detectron2 DefaultPredictor.")

# --- 3. Tahminleri Alıp Yeni Annotation Dosyası Oluşturma ---
# Test veri setini Detectron2 formatında yükleyin
test_dicts = DatasetCatalog.get("my_dataset_test")

new_coco_annotations = {
    "info": {"description": "Annotations generated by Detectron2 model with score threshold 0.3"},
    "licenses": [],
    "images": [],
    "annotations": [],
    "categories": []
}

# Kategori listesini COCO formatına uygun şekilde oluşturun
for idx, name in enumerate(class_names):
    new_coco_annotations['categories'].append({"id": idx + 1, "name": name, "supercategory": "none"})

annotation_id_counter = 1

# Tahminleri al ve yeni annotation'ları oluştur
for d in test_dicts:
    img_path = d["file_name"]
    img_id = d["image_id"]
    height, width = d["height"], d["width"]

    # Görüntü dosyası yolunu tam olarak oluştur
    full_img_path = os.path.join(data_dir, img_path)

    if not os.path.exists(full_img_path):
        print(f"Warning: Image file not found at {full_img_path}. Skipping...")
        continue

    print(f"Processing image: {img_path}")

    try:
        # Predictor ile tahmin yapın
        im = Image.open(full_img_path).convert("RGB")
        outputs = predictor(im)

        instances = outputs["instances"].to("cpu")
        boxes = instances.pred_boxes.tensor.numpy()
        scores = instances.scores.numpy()
        labels = instances.pred_classes.numpy()

        # Resim bilgilerini ekleyin
        new_coco_annotations["images"].append({
            "id": img_id,
            "file_name": os.path.basename(full_img_path),
            "width": width,
            "height": height
        })

        # Tahminleri COCO formatına dönüştürün
        for box, score, label in zip(boxes, scores, labels):
            x_min, y_min, x_max, y_max = box
            bbox = [float(x_min), float(y_min), float(x_max - x_min), float(y_max - y_min)]

            # Detectron2'nin 0'dan başlayan label'ına 1 ekleyerek COCO uyumlu category_id oluştur
            coco_category_id = int(label) + 1

            annotation = {
                "id": annotation_id_counter,
                "image_id": img_id,
                "category_id": coco_category_id,
                "bbox": bbox,
                "area": float((x_max - x_min) * (y_max - y_min)),
                "iscrowd": 0,
                "score": float(score)
            }
            new_coco_annotations["annotations"].append(annotation)
            annotation_id_counter += 1

    except Exception as e:
        print(f"Error processing image {img_path}: {e}")
        continue

# --- 4. Yeni Annotation Dosyasını Kaydetme ---
with open(output_annotations_path, 'w') as f:
    json.dump(new_coco_annotations, f, indent=4)

print(f"\nNew COCO annotations file created at: {output_annotations_path}")

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data import MetadataCatalog
from detectron2.utils.logger import setup_logger
from detectron2.model_zoo import get_config_file, get_checkpoint_url
import os

setup_logger()

def register_my_dataset():
    from detectron2.data.datasets import register_coco_instances

    register_coco_instances(
        "tekstil_train_tek3", {},
        "#your file path/train/_annotations.coco.json",
        "#your file path/train"
    )
    register_coco_instances(
        "tekstil_val_tek3", {},
        "#your file path/valid/_annotations.coco.json",
        "/#your file path/valid"
    )

register_my_dataset()

cfg = get_cfg()
cfg.merge_from_file(get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("tekstil_train_tek3",)
cfg.DATASETS.TEST = ("tekstil_val_tek3",)  # veya () boş olabilir

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # önceden eğitilmiş ağırlıklar
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 5000  # ihtiyacına göre arttır
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8  # senin sınıf sayın

metadata = MetadataCatalog.get("tekstil_train")
metadata.thing_classes = [
    "defect",
    "1- patlak",
    "2- igne_kirigi",
    "3- jut",
    "5- likra_kacigi",
    "6- yag_lekesi",
    "8- May cizgisi",
    "fsa"
]

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

model_path = os.path.join(cfg.OUTPUT_DIR, "model3_final.pth")
trainer.checkpointer.save("model3_final")
print(f"Model {model_path} olarak kaydedildi.")


In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
from detectron2.utils.logger import setup_logger
from detectron2.model_zoo import get_config_file, get_checkpoint_url
import os

# Logger ayarla
setup_logger()

# 1. Datasetleri register et
def register_datasets():
    register_coco_instances(
        "tekstil_train_v3_teks33", {},
        "/#your file path/train/_annotations.coco.json",
        "/#your file path/train"
    )
    register_coco_instances(
        "tekstil_val_v3_teks33", {},
        "/#your file path/valid/_annotations.coco.json",
        "/#your file path/valid"
    )

register_datasets()

# 2. Sınıf isimleri (metadata için)
class_names = [
    "defect",
    "1- patlak",
    "2- igne_kirigi",
    "3- jut",
    "5- likra_kacigi",
    "6- yag_lekesi",
    "8- May cizgisi",
    "fsa"
]

# Metadata'ya sınıfları ekle
MetadataCatalog.get("tekstil_train_v3_teks33").thing_classes = class_names
MetadataCatalog.get("tekstil_val_v3_teks33").thing_classes = class_names

# 3. Config ayarla
cfg = get_cfg()
cfg.merge_from_file(get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("tekstil_train_v3_teks33",)
cfg.DATASETS.TEST = ("tekstil_val_v3_teks33",)  # ya da ()

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # önceden eğitilmiş ağırlıklar
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 5000  # ihtiyaca göre artırabilirsin
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)

# Çıktı dosyalarının kaydedileceği klasör
cfg.OUTPUT_DIR = "/#your file path/model_teks3_v2"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# 4. Trainer oluştur ve eğitimi başlat
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# 5. Eğitim sonunda modeli kaydet
model_path = os.path.join(cfg.OUTPUT_DIR, "model3_final.pth")
trainer.checkpointer.save("model3_final")
print(f"Model {model_path} olarak kaydedildi.")


In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
from detectron2.utils.logger import setup_logger
from detectron2.model_zoo import get_config_file, get_checkpoint_url
import os

# Logger ayarla
setup_logger()

# 1. Datasetleri register et
def register_datasets():
    register_coco_instances(
        "tekstil_train_v5", {},
        "/#your file path/train/_annotations.coco.json",
        "/#your file path/train"
    )
    register_coco_instances(
        "tekstil_val_v5", {},
        "/#your file path/valid/_annotations.coco.json",
        "/#your file path/valid"
    )

register_datasets()

# 2. Sınıf isimleri
class_names = [
    "defect",
    "1- patlak",
    "2- igne_kirigi",
    "3- jut",
    "5- likra_kacigi",
    "6- yag_lekesi",
    "8- May cizgisi",
    "fsa"
]

# Metadata'ya sınıfları ekle
MetadataCatalog.get("tekstil_train_v5").thing_classes = class_names
MetadataCatalog.get("tekstil_val_v5").thing_classes = class_names

# 3. Config ayarla
cfg = get_cfg()
cfg.merge_from_file(get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("tekstil_train_v5",)
cfg.DATASETS.TEST = ("tekstil_val_v5",)  # istersen boş bırakabilirsin: ()

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # önceden eğitilmiş ağırlıklar
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 8000  # ihtiyacına göre artırabilirsin
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)

# 4. Çıktı klasörü
cfg.OUTPUT_DIR = "#your file path/model_teks_v5"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# 5. Trainer oluştur ve eğitimi başlat
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# 6. Model ağırlıklarını kaydet
model_path = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
trainer.checkpointer.save("model_final")
print(f"Model {model_path} olarak kaydedildi.")

# 7. Config dosyasını yaml olarak kaydet
config_path = os.path.join(cfg.OUTPUT_DIR, "config.yaml")
with open(config_path, "w") as f:
    f.write(cfg.dump())
print(f"Config dosyası {config_path} olarak kaydedildi.")


In [ ]:
#KESİN  ANALİZ
import os
import cv2
import matplotlib.pyplot as plt
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# 1. Sınıf isimleri
class_names = [
    "defect",
    "1- patlak",
    "2- igne_kirigi",
    "3- jut",
    "5- likra_kacigi",
    "6- yag_lekesi",
    "8- May cizgisi",
    "fsa"
]

# 2. Metadata ayarı
dataset_name = "tekstil_val_v5"
MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# 3. Config ve model yükle
cfg = get_cfg()
cfg.merge_from_file("/#your file path/model_teks_v5/config.yaml")
cfg.MODEL.WEIGHTS = "/#your file path/model_teks_v5/model_final.pth"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = "cuda" if cv2.cuda.getCudaEnabledDeviceCount() > 0 else "cpu"

predictor = DefaultPredictor(cfg)

# 4. Test klasöründeki görselleri analiz et
test_folder = "/#your file path/tekstil3test/train"
image_files = [f for f in os.listdir(test_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

for img_name in image_files:
    img_path = os.path.join(test_folder, img_name)
    img = cv2.imread(img_path)
    if img is None:
        print(f"Yüklenemedi: {img_name}")
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")

    # Görselleştir (renk kontrastını değiştirmeden)
    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    img_result = out.get_image()[:, :, ::-1]  # BGR formatında

    # Göster
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
    plt.title(img_name)
    plt.axis("off")
    plt.show()


In [ ]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# === 1. Paths ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"
test_images_dir = "/#your file path/train"
annotations_path = os.path.join(test_images_dir, "_annotations_fixed2.coco.json")
output_dir = "/#your file path/test_outputs_confusion_final2"
os.makedirs(output_dir, exist_ok=True)

# === 2. Class names ===
class_names = [
    "defect",
    "1- patlak",
    "2- igne_kirigi",
    "3- jut",
    "5- likra_kacigi",
    "6- yag_lekesi",
    "8- May cizgisi",
    "fsa"
]

# === 3. Metadata ===
dataset_name = "tekstil_val_v5"
MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# === 4. Config setup ===
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
cfg.MODEL.DEVICE = "cuda" if cv2.cuda.getCudaEnabledDeviceCount() > 0 else "cpu"

predictor = DefaultPredictor(cfg)

# === 5. Load annotations ===
with open(annotations_path, "r") as f:
    coco_ann = json.load(f)

filename_to_id = {img["file_name"]: img["id"] for img in coco_ann["images"]}
gt_map = {}
for ann in coco_ann["annotations"]:
    image_id = ann["image_id"]
    cat_id = ann["category_id"]
    gt_map.setdefault(image_id, []).append(cat_id)

# === 6. Prediction & comparison ===
y_true, y_pred = [], []

for file in tqdm(os.listdir(test_images_dir)):
    if not file.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    image_path = os.path.join(test_images_dir, file)
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(file)
    gt_classes = gt_map.get(image_id, [])

    for gt_class in gt_classes:
        if pred_classes:
            y_true.append(gt_class)
            y_pred.append(pred_classes[0])
        else:
            y_true.append(gt_class)
            y_pred.append(-1)  # no prediction

    # Save annotated image
    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]
    save_path = os.path.join(output_dir, file)
    cv2.imwrite(save_path, out_img)

# === 7. Confusion Matrix ===
labels = list(range(len(class_names))) + [-1]
cm = confusion_matrix(y_true, y_pred, labels=labels)
disp_labels = class_names + ["None"]

fig, ax = plt.subplots(figsize=(10, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=disp_labels)
disp.plot(include_values=True, xticks_rotation=45, ax=ax, cmap="Blues")
plt.title("Confusion Matrix")
plt.tight_layout()
cm_path = os.path.join(output_dir, "confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

# === 8. Classification Report ===
report = classification_report(
    y_true, y_pred, labels=labels,
    target_names=disp_labels,
    zero_division=0
)
report_path = os.path.join(output_dir, "classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)


100%|██████████| 129/129 [02:46<00:00,  1.29s/it]


In [ ]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# === 1. Paths ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"
test_images_dir = "/#your file path/train"
# Orijinal ground truth (doğru etiket) dosyasını kullanın
annotations_path = os.path.join(test_images_dir, "_annotations_fixed.coco.json")
output_dir = "/#your file path/test_outputs_confusion_final4"
os.makedirs(output_dir, exist_ok=True)

# === 2. Class names ===
# class_names listesi, Detectron2'nin label_id'sine (0, 1, 2...) karşılık gelir
# Fakat COCO'da label_id'ler genellikle 1'den başlar.
# Kodun içindeki logic bu durumu dikkate alacaktır.
class_names = [
    "defect",
    "1- patlak",
    "2- igne_kirigi",
    "3- jut",
    "5- likra_kacigi",
    "6- yag_lekesi",
    "8- May cizgisi",
    "fsa"
]

# === 3. Metadata ===
dataset_name = "tekstil_val_v5"
MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# === 4. Config setup ===
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3
cfg.MODEL.DEVICE = "cuda" if cv2.cuda.getCudaEnabledDeviceCount() > 0 else "cpu"

predictor = DefaultPredictor(cfg)

# === 5. Load annotations ===
with open(annotations_path, "r") as f:
    coco_ann = json.load(f)

filename_to_id = {img["file_name"]: img["id"] for img in coco_ann["images"]}
gt_map = {}
for ann in coco_ann["annotations"]:
    image_id = ann["image_id"]
    # COCO'daki etiketler 1'den başladığı için, 0'dan başlayan model etiketleriyle
    # eşleştirmek için bir harita oluşturulabilir veya 1 çıkarılabilir
    cat_id = ann["category_id"]
    gt_map.setdefault(image_id, []).append(cat_id)

# === 6. Prediction & comparison ===
y_true, y_pred = [], []
image_files = [f for f in os.listdir(test_images_dir) if f in filename_to_id]

for file in tqdm(image_files):
    image_path = os.path.join(test_images_dir, file)
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")

    pred_classes = instances.pred_classes.numpy().tolist()
    image_id = filename_to_id[file]
    gt_classes = gt_map.get(image_id, [])

    # Tahminlerin ve ground truth'ların eşleşmesi için basitleştirilmiş bir döngü
    # Her bir ground truth için bir tahmin eşleştirilmeye çalışılıyor
    for i, gt_class_id in enumerate(gt_classes):
        # Ground truth etiketini y_true'ya ekle
        y_true.append(gt_class_id)

        # Eğer bir tahmin varsa, ilk tahmin edilen sınıfı kullan
        if i < len(pred_classes):
            # Model 0'dan başlayan etiketler ürettiği için +1 ekliyoruz
            y_pred.append(pred_classes[i] + 1)
        else:
            # Tahmin yoksa, "None" olarak işaretle
            y_pred.append(-1)

    # Eğer ground truth'tan daha fazla tahmin varsa, kalan tahminleri de ekle
    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2) # Ekstra tahmin için özel bir etiket
        y_pred.append(pred_classes[i] + 1)

    # Save annotated image
    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]
    save_path = os.path.join(output_dir, file)
    cv2.imwrite(save_path, out_img)

# === 7. Confusion Matrix ===
# Etiketleri, 1'den başlayan sınıf ID'leri ve -1 (None) ile oluştur
labels = list(range(1, len(class_names) + 1)) + [-1]
disp_labels = class_names + ["None"]

# Sınıflandırma raporu ve karmaşıklık matrisi için gerekli etiketleri hazırlayın
# Eğer y_true veya y_pred'de -2 etiketi varsa, bunu da dikkate alın
cm = confusion_matrix(y_true, y_pred, labels=labels)

fig, ax = plt.subplots(figsize=(12, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=disp_labels)
disp.plot(include_values=True, xticks_rotation=45, ax=ax, cmap="Blues")
plt.title("Confusion Matrix")
plt.tight_layout()
cm_path = os.path.join(output_dir, "confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

# === 8. Classification Report ===
report = classification_report(
    y_true, y_pred, labels=labels,
    target_names=disp_labels,
    zero_division=0
)
report_path = os.path.join(output_dir, "classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)
print(f"Confusion Matrix and Classification Report saved to {output_dir}")
print("Script finished successfully.")

In [ ]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import load_coco_json

# === 1. Dosya Yolları ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"

# İki ayrı test veri yolu ve annotation dosyası
test_dir_1 = "/#your file path/train"
annotations_path_1 = os.path.join(test_dir_1, "_annotations_fixed2.coco.json")

test_dir_2 = "/#your file path/test"
annotations_path_2 = os.path.join(test_dir_2, "_annotations.coco.json")

output_dir = "/#your file path/combined_test_outputs"
os.makedirs(output_dir, exist_ok=True)

# === 2. Sınıf İsimleri ===
class_names = [
    "defect",
    "1- patlak",
    "2- igne_kirigi",
    "3- jut",
    "5- likra_kacigi",
    "6- yag_lekesi",
    "8- May cizgisi",
    "fsa"
]

# === 3. Metadata ve Config ===
dataset_name = "tekstil_combined_test"
# Toplam veri seti için tek bir meta veri nesnesi oluşturun
MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# Modeli ve config'i yükleyin
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("Model loaded with Detectron2 DefaultPredictor.")

# === 4. Birleşik Ground Truth Verilerini Yükleme ===
# Her iki veri setinin etiketlerini ve görsel yollarını birleştirin
combined_dicts = []
combined_dicts.extend(load_coco_json(annotations_path_1, test_dir_1))
combined_dicts.extend(load_coco_json(annotations_path_2, test_dir_2))

# === 5. Tahmin ve Karşılaştırma ===
y_true, y_pred = [], []
image_files_with_paths = []

for d in combined_dicts:
    # `d["file_name"]` zaten tam yolu içeriyor
    image_files_with_paths.append(d["file_name"])

for image_path in tqdm(image_files_with_paths):
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    # Ground truth sınıflarını yüklenen verilerden alın
    # COCO'da kategori ID'leri 1'den başlar, bu yüzden 1'den başlayan ID'ler y_true'ya eklenir
    gt_classes = [ann['category_id'] for ann in d.get('annotations', []) if 'category_id' in ann]

    # Tahminlerin ve ground truth'ların eşleşmesi için basit döngü
    # Her bir ground truth için bir tahmin eşleştirilir.
    for i in range(len(gt_classes)):
        y_true.append(gt_classes[i])

        if i < len(pred_classes):
            # Model 0'dan başlayan etiketler ürettiği için +1 ekliyoruz (COCO ID'si için)
            y_pred.append(pred_classes[i] + 1)
        else:
            y_pred.append(-1) # Tahmin yoksa, "None" olarak işaretle

    # Fazla tahminleri (False Positives) ekle
    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2) # Ekstra tahmin için özel bir etiket
        y_pred.append(pred_classes[i] + 1)

    # Görseli kaydet
    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]

    # Çıktı dosya adı
    save_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(save_path, out_img)

# === 6. Karmaşıklık Matrisi ve Raporu ===
labels = list(range(1, len(class_names) + 1)) + [-1]
disp_labels = class_names + ["None"]

cm = confusion_matrix(y_true, y_pred, labels=labels)

fig, ax = plt.subplots(figsize=(12, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=disp_labels)
disp.plot(include_values=True, xticks_rotation=45, ax=ax, cmap="Blues")
plt.title("Combined Confusion Matrix")
plt.tight_layout()
cm_path = os.path.join(output_dir, "combined_confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

# Sınıflandırma raporu
report = classification_report(
    y_true, y_pred, labels=labels,
    target_names=disp_labels,
    zero_division=0
)
report_path = os.path.join(output_dir, "combined_classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)

print(f"Combined Confusion Matrix and Classification Report saved to {output_dir}")

In [ ]:
#başarısız confusion matrix hesabı yaptı
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer

# === 1. Paths ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"
test_images_dir = "/#your file path/train"
annotations_path = os.path.join(test_images_dir, "_annotations_fixed.coco.json")
output_dir = "/#your file path/test_outputs"
os.makedirs(output_dir, exist_ok=True)

# === 2. Class names ===
class_names = [
    "defect",
    "1- patlak",
    "2- igne_kirigi",
    "3- jut",
    "5- likra_kacigi",
    "6- yag_lekesi",
    "8- May cizgisi",
    "fsa"
]

# === 3. Metadata ===
dataset_name = "tekstil_val_v5"
MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# === 4. Config setup ===
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3
cfg.MODEL.DEVICE = "cuda" if cv2.cuda.getCudaEnabledDeviceCount() > 0 else "cpu"

predictor = DefaultPredictor(cfg)

# === 5. Load annotations ===
with open(annotations_path, "r") as f:
    coco_ann = json.load(f)

gt_map = {}
for ann in coco_ann["annotations"]:
    image_id = ann["image_id"]
    cat_id = ann["category_id"]
    gt_map.setdefault(image_id, []).append(cat_id)

filename_to_id = {img["file_name"]: img["id"] for img in coco_ann["images"]}

# === 6. Prediction & saving ===
y_true, y_pred = [], []

for file in tqdm(os.listdir(test_images_dir)):
    if not file.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    image_path = os.path.join(test_images_dir, file)
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(file)
    gt_classes = gt_map.get(image_id, [])

    for gt_class in gt_classes:
        if pred_classes:
            y_true.append(gt_class)
            y_pred.append(pred_classes[0])
        else:
            y_true.append(gt_class)
            y_pred.append(-1)  # No prediction

    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]
    save_path = os.path.join(output_dir, file)
    cv2.imwrite(save_path, out_img)

# === 7. Confusion Matrix ===
labels = list(range(len(class_names)))
cm = confusion_matrix(y_true, y_pred, labels=labels + [-1])

fig, ax = plt.subplots(figsize=(10, 8))


In [ ]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# === 1. Dosya Yolları ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"

# İki ayrı test veri yolu
test_dir_1 = "/#your file path/train"
annotations_path_1 = os.path.join(test_dir_1, "_annotations_fixed2.coco.json")

test_dir_2 = "/#your file path/test"
annotations_path_2 = os.path.join(test_dir_2, "_annotations.coco.json")

output_dir = "/#your file path/combined_test_outputs33"
os.makedirs(output_dir, exist_ok=True)

# === 2. Sınıf İsimleri ===
class_names = [
    "defect",
    "1- patlak",
    "2- igne_kirigi",
    "3- jut",
    "5- likra_kacigi",
    "6- yag_lekesi",
    "8- May cizgisi",
    "fsa"
]

# === 3. Metadata ve Config ===
dataset_name = "tekstil_combined_test"
MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("Model loaded with Detectron2 DefaultPredictor.")

# === 4. Birleşik Ground Truth Verilerini Yükleme ===
gt_map = {}
filename_to_id = {}
all_image_paths = []

def load_annotations_and_images(annotations_path, image_dir):
    global gt_map, filename_to_id, all_image_paths

    if not os.path.exists(annotations_path):
        print(f"Warning: Annotation file not found at {annotations_path}. Skipping this directory.")
        return

    with open(annotations_path, "r") as f:
        coco_ann = json.load(f)
        for img in coco_ann["images"]:
            filename = img["file_name"]
            img_id = img["id"]
            full_path = os.path.join(image_dir, filename)

            # Eğer aynı ID'ye sahip bir dosya zaten işlenmişse, atla
            if img_id in gt_map:
                continue

            filename_to_id[filename] = img_id
            all_image_paths.append(full_path)
        for ann in coco_ann["annotations"]:
            gt_map.setdefault(ann["image_id"], []).append(ann["category_id"])

load_annotations_and_images(annotations_path_1, test_dir_1)
load_annotations_and_images(annotations_path_2, test_dir_2)

# === 5. Tahmin ve Karşılaştırma ===
y_true, y_pred = [], []

print(f"\nAnalyzing {len(all_image_paths)} images...")
for image_path in tqdm(all_image_paths):
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(os.path.basename(image_path))
    gt_classes = gt_map.get(image_id, [])

    # Tahminlerin ve ground truth'ların eşleşmesi için basitleştirilmiş döngü
    for i in range(len(gt_classes)):
        y_true.append(gt_classes[i])

        if i < len(pred_classes):
            # Model 0'dan başlayan etiketler ürettiği için +1 ekliyoruz (COCO ID'si için)
            y_pred.append(pred_classes[i] + 1)
        else:
            y_pred.append(-1)

    # Fazla tahminleri (False Positives) de ekle
    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2)
        y_pred.append(pred_classes[i] + 1)

    # Görseli kaydet
    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]

    save_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(save_path, out_img)

# === 6. Karmaşıklık Matrisi ve Raporu ===
labels = list(range(1, len(class_names) + 1)) + [-1]
disp_labels = class_names + ["None"]

cm = confusion_matrix(y_true, y_pred, labels=labels)

fig, ax = plt.subplots(figsize=(12, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=disp_labels)
disp.plot(include_values=True, xticks_rotation=45, ax=ax, cmap="Blues")
plt.title("Combined Confusion Matrix")
plt.tight_layout()
cm_path = os.path.join(output_dir, "combined_confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

# Sınıflandırma raporu
report = classification_report(
    y_true, y_pred, labels=labels,
    target_names=disp_labels,
    zero_division=0
)
report_path = os.path.join(output_dir, "combined_classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)

print(f"\nCombined Confusion Matrix and Classification Report saved to {output_dir}")

In [ ]:

import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# === 1. Dosya Yolları ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"

# İki ayrı test veri yolu
test_dir_1 = "/#your file path/train"
annotations_path_1 = os.path.join(test_dir_1, "_annotations_fixed2.coco.json")

test_dir_2 = "/#your file path/test"
annotations_path_2 = os.path.join(test_dir_2, "_annotations.coco.json")

output_dir = "/#your file path/combined_test_outputs33"
os.makedirs(output_dir, exist_ok=True)

# === 2. Sınıf İsimleri ===
class_names = [
    "defect",
    "1- patlak",
    "2- igne_kirigi",
    "3- jut",
    "5- likra_kacigi",
    "6- yag_lekesi",
    "8- May cizgisi",
    "fsa"
]

# === 3. Metadata ve Config ===
dataset_name = "tekstil_combined_test"
MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("Model loaded with Detectron2 DefaultPredictor.")

# === 4. Birleşik Ground Truth Verilerini Yükleme ===
gt_map = {}
filename_to_id = {}
all_image_paths = []

def load_annotations_and_images(annotations_path, image_dir):
    global gt_map, filename_to_id, all_image_paths

    if not os.path.exists(annotations_path):
        print(f"Warning: Annotation file not found at {annotations_path}. Skipping this directory.")
        return

    with open(annotations_path, "r") as f:
        coco_ann = json.load(f)
        for img in coco_ann["images"]:
            filename = img["file_name"]
            img_id = img["id"]
            full_path = os.path.join(image_dir, filename)

            # Eğer aynı ID'ye sahip bir dosya zaten işlenmişse, atla
            if img_id in gt_map:
                continue

            filename_to_id[filename] = img_id
            all_image_paths.append(full_path)
        for ann in coco_ann["annotations"]:
            gt_map.setdefault(ann["image_id"], []).append(ann["category_id"])

load_annotations_and_images(annotations_path_1, test_dir_1)
load_annotations_and_images(annotations_path_2, test_dir_2)

# === 5. Tahmin ve Karşılaştırma ===
y_true, y_pred = [], []

print(f"\nAnalyzing {len(all_image_paths)} images...")
for image_path in tqdm(all_image_paths):
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(os.path.basename(image_path))
    gt_classes = gt_map.get(image_id, [])

    # Tahminlerin ve ground truth'ların eşleşmesi için basitleştirilmiş döngü
    for i in range(len(gt_classes)):
        y_true.append(gt_classes[i])

        if i < len(pred_classes):
            y_pred.append(pred_classes[i] + 1)
        else:
            y_pred.append(-1)

    # Fazla tahminleri (False Positives) de ekle
    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2)
        y_pred.append(pred_classes[i] + 1)

    # Görseli kaydet
    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]

    save_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(save_path, out_img)

# === 6. Karmaşıklık Matrisi ve Raporu ===
labels = list(range(1, len(class_names) + 1)) + [-1]
disp_labels = class_names + ["None"]

cm = confusion_matrix(y_true, y_pred, labels=labels)

fig, ax = plt.subplots(figsize=(12, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=disp_labels)
disp.plot(include_values=True, xticks_rotation=45, ax=ax, cmap="Blues")
plt.title("Combined Confusion Matrix")
plt.tight_layout()
cm_path = os.path.join(output_dir, "combined_confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

# Sınıflandırma raporu
report = classification_report(
    y_true, y_pred, labels=labels,
    target_names=disp_labels,
    zero_division=0
)
report_path = os.path.join(output_dir, "combined_classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)

print(f"\nCombined Confusion Matrix and Classification Report saved to {output_dir}")

In [ ]:
import os
import shutil
import json
from tqdm import tqdm

# === 1. Dosya Yolları ===
# Birleştirilecek ilk veri seti klasörü ve annotation dosyası
source_dir_1 = "/#your file path/train"
annotations_path_1 = os.path.join(source_dir_1, "_annotations_fixed.coco.json")

# Birleştirilecek ikinci veri seti klasörü ve annotation dosyası
source_dir_2 = "/#your file path/test"
annotations_path_2 = os.path.join(source_dir_2, "_annotations.coco.json")

# Birleştirilmiş verilerin kaydedileceği hedef klasör
combined_output_dir = "/#your file path/combined_dataset"
combined_annotations_path = os.path.join(combined_output_dir, "combined_annotations.coco.json")

# Hedef klasörü oluştur (varsa silip yeniden oluşturur)
if os.path.exists(combined_output_dir):
    shutil.rmtree(combined_output_dir)
os.makedirs(combined_output_dir, exist_ok=True)

# --- 2. Birleştirme Fonksiyonu ---
def combine_coco_datasets(paths):
    combined_images = []
    combined_annotations = []
    combined_categories = []

    image_id_offset = 0
    ann_id_offset = 0
    category_id_map = {}

    print("Veri setleri birleştiriliyor...")

    for ann_path, img_dir in paths:
        if not os.path.exists(ann_path):
            print(f"Uyarı: Annotation dosyası bulunamadı, atlanıyor: {ann_path}")
            continue

        with open(ann_path, 'r') as f:
            data = json.load(f)

        # Kategorileri harita kullanarak ekle veya güncelle
        for cat in data['categories']:
            if cat['name'] not in category_id_map:
                new_id = len(category_id_map) + 1
                category_id_map[cat['name']] = new_id
                combined_categories.append({'id': new_id, 'name': cat['name'], 'supercategory': cat.get('supercategory', 'none')})

        # Resim ID'lerini ve annotation ID'lerini güncelle ve kopyala
        image_id_mapping = {}
        for image in tqdm(data['images'], desc=f"Görseller kopyalanıyor: {img_dir}"):
            old_image_id = image['id']
            new_image_id = old_image_id + image_id_offset
            image_id_mapping[old_image_id] = new_image_id

            image_copy = image.copy()
            image_copy['id'] = new_image_id

            src_image_path = os.path.join(img_dir, image['file_name'])
            dst_image_path = os.path.join(combined_output_dir, image['file_name'])
            if os.path.exists(src_image_path):
                shutil.copy(src_image_path, dst_image_path)

            combined_images.append(image_copy)

        for annotation in data['annotations']:
            ann_copy = annotation.copy()
            ann_copy['id'] = annotation['id'] + ann_id_offset
            ann_copy['image_id'] = image_id_mapping[annotation['image_id']]

            # Kategori ID'lerini yeni haritaya göre güncelle
            original_cat_name = next(cat['name'] for cat in data['categories'] if cat['id'] == annotation['category_id'])
            ann_copy['category_id'] = category_id_map[original_cat_name]

            combined_annotations.append(ann_copy)

        # Bir sonraki veri seti için offset'leri güncelle
        image_id_offset += len(data['images'])
        ann_id_offset += len(data['annotations'])

    combined_data = {
        "info": {"description": "Combined COCO dataset"},
        "licenses": data.get("licenses", []),
        "images": combined_images,
        "annotations": combined_annotations,
        "categories": combined_categories
    }

    return combined_data

# --- 3. Birleştirme İşlemini Başlat ---
paths_to_combine = [
    (annotations_path_1, source_dir_1),
    (annotations_path_2, source_dir_2)
]

combined_dataset = combine_coco_datasets(paths_to_combine)

with open(combined_annotations_path, "w") as f:
    json.dump(combined_dataset, f, indent=4)

print(f"\nBirleştirme tamamlandı. Görseller ve 'combined_annotations.coco.json' dosyası buraya kaydedildi: {combined_output_dir}")

In [ ]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data.datasets import register_coco_instances

# === 1. Dosya Yolları ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"

# Birleştirilmiş veri setinin yolu
combined_dataset_dir = "/#your file path/combined_dataset"
annotations_path = os.path.join(combined_dataset_dir, "combined_annotations.coco.json")

# Çıktıların kaydedileceği klasör
output_dir = "/#your file path/combined_analysis_report"
os.makedirs(output_dir, exist_ok=True)

# === 2. Sınıf İsimleri ===
class_names = [
    "defect",
    "1- patlak",
    "2- igne_kirigi",
    "3- jut",
    "5- likra_kacigi",
    "6- yag_lekesi",
    "8- May cizgisi",
    "fsa"
]

# === 3. Metadata ve Config ===
dataset_name = "tekstil_combined_dataset"
try:
    register_coco_instances(dataset_name, {}, annotations_path, combined_dataset_dir)
except ValueError:
    # Veri seti zaten kayıtlıysa hata vermesini engelle
    pass

MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2  # Tahmin eşiği
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("Model loaded with Detectron2 DefaultPredictor.")

# === 4. Ground Truth Verilerini Yükleme ===
gt_map = {}
filename_to_id = {}
all_image_paths = []

with open(annotations_path, "r") as f:
    coco_ann = json.load(f)
    for img in coco_ann["images"]:
        filename = img["file_name"]
        img_id = img["id"]
        full_path = os.path.join(combined_dataset_dir, filename)
        filename_to_id[filename] = img_id
        all_image_paths.append(full_path)
    for ann in coco_ann["annotations"]:
        gt_map.setdefault(ann["image_id"], []).append(ann["category_id"])

# === 5. Tahmin ve Karşılaştırma ===
y_true, y_pred = [], []

print(f"\nAnalyzing {len(all_image_paths)} images...")
for image_path in tqdm(all_image_paths):
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(os.path.basename(image_path))
    gt_classes = gt_map.get(image_id, [])

    # Tahminlerin ve ground truth'ların eşleşmesi için basitleştirilmiş döngü
    # Not: Bu, her bir ground truth için bir tahmin eşleştirmeye çalışır.
    for i in range(len(gt_classes)):
        y_true.append(gt_classes[i])

        if i < len(pred_classes):
            # Model 0'dan başlayan etiketler ürettiği için +1 ekliyoruz (COCO ID'si için)
            y_pred.append(pred_classes[i] + 1)
        else:
            y_pred.append(-1) # Tahmin yoksa, "None" olarak işaretle

    # Fazla tahminleri (False Positives) de ekle
    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2) # Ekstra tahmin için özel bir etiket
        y_pred.append(pred_classes[i] + 1)

    # Görseli tahminlerle kaydet
    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]

    save_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(save_path, out_img)

# === 6. Karmaşıklık Matrisi ve Raporu ===
# Etiketleri, 1'den başlayan sınıf ID'leri ve -1 (None) ile oluştur
labels = list(range(1, len(class_names) + 1)) + [-1]
disp_labels = class_names + ["None"]

cm = confusion_matrix(y_true, y_pred, labels=labels)

fig, ax = plt.subplots(figsize=(12, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=disp_labels)
disp.plot(include_values=True, xticks_rotation=45, ax=ax, cmap="Blues")
plt.title("Combined Confusion Matrix")
plt.tight_layout()
cm_path = os.path.join(output_dir, "combined_confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

# Sınıflandırma raporu
report = classification_report(
    y_true, y_pred, labels=labels,
    target_names=disp_labels,
    zero_division=0
)
report_path = os.path.join(output_dir, "combined_classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)

print(f"\nAnaliz tamamlandı. Raporlar ve görseller buraya kaydedildi: {output_dir}")

In [ ]:
import os
import shutil
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

# === 1. Dosya Yolları ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"

# Kaynak veri setlerinin yolları
source_dir_1 = "/#your file path/train"
annotations_path_1 = os.path.join(source_dir_1, "_annotations_fixed2.coco.json")
source_dir_2 = "/#your file path/test"
annotations_path_2 = os.path.join(source_dir_2, "_annotations.coco.json")

# Birleştirilmiş verilerin ve çıktıların kaydedileceği ana klasörler
combined_dataset_dir = "/#your file path/combined_dataset"
combined_annotations_path = os.path.join(combined_dataset_dir, "combined_annotations.coco.json")
output_dir = "/#your file path/combined_analysis_report3"

# Klasörleri oluştur/temizle
if os.path.exists(combined_dataset_dir):
    shutil.rmtree(combined_dataset_dir)
os.makedirs(combined_dataset_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# === 2. Veri Kümelerini Birleştirme Fonksiyonu ===
def combine_coco_datasets(paths, output_ann_path, output_img_dir):
    combined_images = []
    combined_annotations = []
    combined_categories = []

    image_id_offset = 0
    ann_id_offset = 0
    category_id_map = {}

    print("Veri setleri birleştiriliyor...")

    for ann_path, img_dir in paths:
        if not os.path.exists(ann_path):
            print(f"Uyarı: Annotation dosyası bulunamadı, atlanıyor: {ann_path}")
            continue

        with open(ann_path, 'r') as f:
            data = json.load(f)

        for cat in data['categories']:
            if cat['name'] not in category_id_map:
                new_id = len(category_id_map) + 1
                category_id_map[cat['name']] = new_id
                combined_categories.append({'id': new_id, 'name': cat['name'], 'supercategory': cat.get('supercategory', 'none')})

        image_id_mapping = {}
        for image in tqdm(data['images'], desc=f"Görseller kopyalanıyor: {img_dir}"):
            old_image_id = image['id']
            new_image_id = old_image_id + image_id_offset
            image_id_mapping[old_image_id] = new_image_id

            image_copy = image.copy()
            image_copy['id'] = new_image_id

            src_image_path = os.path.join(img_dir, image['file_name'])
            dst_image_path = os.path.join(output_img_dir, image['file_name'])
            if os.path.exists(src_image_path):
                shutil.copy(src_image_path, dst_image_path)

            combined_images.append(image_copy)

        for annotation in data['annotations']:
            ann_copy = annotation.copy()
            ann_copy['id'] = annotation['id'] + ann_id_offset
            ann_copy['image_id'] = image_id_mapping[annotation['image_id']]

            original_cat_name = next(cat['name'] for cat in data['categories'] if cat['id'] == annotation['category_id'])
            ann_copy['category_id'] = category_id_map[original_cat_name]

            combined_annotations.append(ann_copy)

        image_id_offset += len(data['images'])
        ann_id_offset += len(data['annotations'])

    combined_data = {
        "info": {"description": "Combined COCO dataset"},
        "licenses": [],
        "images": combined_images,
        "annotations": combined_annotations,
        "categories": combined_categories
    }

    with open(output_ann_path, "w") as f:
        json.dump(combined_data, f, indent=4)

    print("\nVeri birleştirme tamamlandı.")

# --- Birleştirme İşlemini Başlat ---
paths_to_combine = [
    (annotations_path_1, source_dir_1),
    (annotations_path_2, source_dir_2)
]
combine_coco_datasets(paths_to_combine, combined_annotations_path, combined_dataset_dir)


# === 3. Analiz için Sınıf İsimleri ve Metadata ===
class_names = [
    "defect",
    "1- patlak",
    "2- igne_kirigi",
    "3- jut",
    "5- likra_kacigi",
    "6- yag_lekesi",
    "8- May cizgisi",
    "fsa"
]

dataset_name = "tekstil_combined_dataset"
try:
    register_coco_instances(dataset_name, {}, combined_annotations_path, combined_dataset_dir)
except ValueError:
    pass

MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# === 4. Model Yükleme ===
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("\nModel loaded with Detectron2 DefaultPredictor.")


# === 5. Tahmin ve Karşılaştırma ===
gt_map = {}
filename_to_id = {}
all_image_paths = []

with open(combined_annotations_path, "r") as f:
    coco_ann = json.load(f)
    for img in coco_ann["images"]:
        filename = img["file_name"]
        img_id = img["id"]
        full_path = os.path.join(combined_dataset_dir, filename)
        filename_to_id[filename] = img_id
        all_image_paths.append(full_path)
    for ann in coco_ann["annotations"]:
        gt_map.setdefault(ann["image_id"], []).append(ann["category_id"])

y_true, y_pred = [], []
print(f"\nAnalyzing {len(all_image_paths)} images...")
for image_path in tqdm(all_image_paths):
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(os.path.basename(image_path))
    gt_classes = gt_map.get(image_id, [])

    for i in range(len(gt_classes)):
        y_true.append(gt_classes[i])
        if i < len(pred_classes):
            y_pred.append(pred_classes[i] + 1)
        else:
            y_pred.append(-1)

    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2)
        y_pred.append(pred_classes[i] + 1)

    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]
    save_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(save_path, out_img)

# === 6. Karmaşıklık Matrisi ve Raporu ===
labels = list(range(1, len(class_names) + 1)) + [-1]
disp_labels = class_names + ["None"]

cm = confusion_matrix(y_true, y_pred, labels=labels)

fig, ax = plt.subplots(figsize=(12, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=disp_labels)
disp.plot(include_values=True, xticks_rotation=45, ax=ax, cmap="Blues")
plt.title("Combined Confusion Matrix")
plt.tight_layout()
cm_path = os.path.join(output_dir, "combined_confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

report = classification_report(
    y_true, y_pred, labels=labels,
    target_names=disp_labels,
    zero_division=0
)
report_path = os.path.join(output_dir, "combined_classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)

print(f"\nAnaliz tamamlandı. Raporlar ve görseller buraya kaydedildi: {output_dir}")

In [ ]:
import os
import shutil
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch
import random
import string
import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances


# === 1. Dosya Yolları ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"

# Kaynak veri setlerinin yolları
source_dir_1 = "/#your file path/train"
annotations_path_1 = os.path.join(source_dir_1, "ikinci.coco.json")
source_dir_2 = "/#your file path/test"
annotations_path_2 = os.path.join(source_dir_2, "_annotations.coco.json")

# Birleştirilmiş verilerin ve çıktıların kaydedileceği ana klasörler
combined_dataset_dir = "/#your file path/combined_dataset"
combined_annotations_path = os.path.join(combined_dataset_dir, "combined_annotations.coco.json")
output_dir = "/#your file path/combined_analysis_report22"

# Klasörleri oluştur/temizle
if os.path.exists(combined_dataset_dir):
    shutil.rmtree(combined_dataset_dir)
os.makedirs(combined_dataset_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# === 2. Veri Kümelerini Birleştirme Fonksiyonu ===
def combine_coco_datasets(paths, output_ann_path, output_img_dir):
    combined_images = []
    combined_annotations = []
    combined_categories = []

    image_id_offset = 0
    ann_id_offset = 0
    category_id_map = {}

    print("Veri setleri birleştiriliyor...")

    for ann_path, img_dir in paths:
        if not os.path.exists(ann_path):
            print(f"Uyarı: Annotation dosyası bulunamadı, atlanıyor: {ann_path}")
            continue

        with open(ann_path, 'r') as f:
            data = json.load(f)

        for cat in data['categories']:
            if cat['name'] not in category_id_map:
                new_id = len(category_id_map) + 1
                category_id_map[cat['name']] = new_id
                combined_categories.append({'id': new_id, 'name': cat['name'], 'supercategory': cat.get('supercategory', 'none')})

        image_id_mapping = {}
        processed_count = 0
        skipped_count = 0

        for image in tqdm(data['images'], desc=f"Görseller kopyalanıyor: {img_dir}"):
            src_image_path = os.path.join(img_dir, image['file_name'])

            if os.path.exists(src_image_path):
                old_image_id = image['id']
                new_image_id = old_image_id + image_id_offset
                image_id_mapping[old_image_id] = new_image_id

                image_copy = image.copy()
                image_copy['id'] = new_image_id

                dst_image_path = os.path.join(output_img_dir, image['file_name'])
                shutil.copy(src_image_path, dst_image_path)

                combined_images.append(image_copy)
                processed_count += 1
            else:
                skipped_count += 1
                print(f"\nUyarı: {src_image_path} bulunamadı, bu görsel atlanıyor.")

        print(f"\n{img_dir} klasöründen {processed_count} görsel başarıyla kopyalandı, {skipped_count} görsel atlandı.")

        for annotation in data['annotations']:
            if annotation['image_id'] in image_id_mapping:
                ann_copy = annotation.copy()
                ann_copy['id'] = annotation['id'] + ann_id_offset
                ann_copy['image_id'] = image_id_mapping[annotation['image_id']]

                original_cat_name = next(cat['name'] for cat in data['categories'] if cat['id'] == annotation['category_id'])
                ann_copy['category_id'] = category_id_map[original_cat_name]

                combined_annotations.append(ann_copy)

        image_id_offset += processed_count
        ann_id_offset += len(data['annotations'])

    combined_data = {
        "info": {"description": "Combined COCO dataset"},
        "licenses": [],
        "images": combined_images,
        "annotations": combined_annotations,
        "categories": combined_categories
    }

    with open(output_ann_path, "w") as f:
        json.dump(combined_data, f, indent=4)

    print("\nVeri birleştirme tamamlandı.")

# --- Birleştirme İşlemini Başlat ---
paths_to_combine = [
    (annotations_path_1, source_dir_1),
    (annotations_path_2, source_dir_2)
]
combine_coco_datasets(paths_to_combine, combined_annotations_path, combined_dataset_dir)


# === 3. Analiz için Sınıf İsimleri ve Metadata ===
class_names = [
    "defect", "1- patlak", "2- igne_kirigi", "3- jut", "5- likra_kacigi", "6- yag_lekesi", "8- May cizgisi", "fsa"
]

unique_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
dataset_name = f"tekstil_combined_dataset_{unique_suffix}"

try:
    register_coco_instances(dataset_name, {}, combined_annotations_path, combined_dataset_dir)
except ValueError:
    pass

MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# === 4. Model Yükleme ===
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("\nModel loaded with Detectron2 DefaultPredictor.")


# === 5. Tahmin ve Karşılaştırma ===
gt_map = {}
filename_to_id = {}
all_image_paths = []

with open(combined_annotations_path, "r") as f:
    coco_ann = json.load(f)
    for img in coco_ann["images"]:
        filename = img["file_name"]
        img_id = img["id"]
        full_path = os.path.join(combined_dataset_dir, filename)
        filename_to_id[filename] = img_id
        all_image_paths.append(full_path)
    for ann in coco_ann["annotations"]:
        gt_map.setdefault(ann["image_id"], []).append(ann["category_id"])

y_true, y_pred = [], []
print(f"\nAnalyzing {len(all_image_paths)} images...")
for image_path in tqdm(all_image_paths):
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(os.path.basename(image_path))
    gt_classes = gt_map.get(image_id, [])

    for i in range(len(gt_classes)):
        y_true.append(gt_classes[i])
        if i < len(pred_classes):
            y_pred.append(pred_classes[i] + 1)
        else:
            y_pred.append(-1)

    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2)
        y_pred.append(pred_classes[i] + 1)

    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]
    save_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(save_path, out_img)

# === 6. Karmaşıklık Matrisi ve Raporu ===
labels = list(range(1, len(class_names) + 1)) + [-1]
disp_labels = class_names + ["None"]

cm = confusion_matrix(y_true, y_pred, labels=labels)

fig, ax = plt.subplots(figsize=(12, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=disp_labels)
disp.plot(include_values=True, xticks_rotation=45, ax=ax, cmap="Blues")
plt.title("Combined Confusion Matrix")
plt.tight_layout()
cm_path = os.path.join(output_dir, "combined_confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

report = classification_report(
    y_true, y_pred, labels=labels,
    target_names=disp_labels,
    zero_division=0
)
report_path = os.path.join(output_dir, "combined_classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)

print(f"\nAnaliz tamamlandı. Raporlar ve görseller buraya kaydedildi: {output_dir}")

In [ ]:
import os
import shutil
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch
import random
import string
import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

# Google Drive'ı bağlama (Colab için)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

# === 1. Dosya Yolları ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"

# Kaynak veri setlerinin yolları
source_dir_1 = "/#your file path/train"
annotations_path_1 = os.path.join(source_dir_1, "ikinci.coco.json")
source_dir_2 = "/#your file path/test"
annotations_path_2 = os.path.join(source_dir_2, "_annotations.coco.json")

# Birleştirilmiş verilerin ve çıktıların kaydedileceği ana klasörler
combined_dataset_dir = "/#your file path/combined_dataset"
combined_annotations_path = os.path.join(combined_dataset_dir, "combined_annotations.coco.json")
output_dir = "/#your file path/combined_analysis_report33"

# Klasörleri oluştur/temizle
if os.path.exists(combined_dataset_dir):
    shutil.rmtree(combined_dataset_dir)
os.makedirs(combined_dataset_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# === 2. Veri Kümelerini Birleştirme Fonksiyonu ===
def combine_coco_datasets(paths, output_ann_path, output_img_dir):
    combined_images = []
    combined_annotations = []
    combined_categories = []

    image_id_offset = 0
    ann_id_offset = 0
    category_id_map = {}

    print("Veri setleri birleştiriliyor...")

    for ann_path, img_dir in paths:
        if not os.path.exists(ann_path):
            print(f"Uyarı: Annotation dosyası bulunamadı, atlanıyor: {ann_path}")
            continue

        with open(ann_path, 'r') as f:
            data = json.load(f)

        for cat in data['categories']:
            if cat['name'] not in category_id_map:
                new_id = len(category_id_map) + 1
                category_id_map[cat['name']] = new_id
                combined_categories.append({'id': new_id, 'name': cat['name'], 'supercategory': cat.get('supercategory', 'none')})

        image_id_mapping = {}
        processed_count = 0
        skipped_count = 0

        for image in tqdm(data['images'], desc=f"Görseller kopyalanıyor: {img_dir}"):
            src_image_path = os.path.join(img_dir, image['file_name'])

            if os.path.exists(src_image_path):
                old_image_id = image['id']
                new_image_id = old_image_id + image_id_offset
                image_id_mapping[old_image_id] = new_image_id

                image_copy = image.copy()
                image_copy['id'] = new_image_id

                dst_image_path = os.path.join(output_img_dir, image['file_name'])
                shutil.copy(src_image_path, dst_image_path)

                combined_images.append(image_copy)
                processed_count += 1
            else:
                skipped_count += 1
                print(f"\nUyarı: {src_image_path} bulunamadı, bu görsel atlanıyor.")

        print(f"\n{img_dir} klasöründen {processed_count} görsel başarıyla kopyalandı, {skipped_count} görsel atlandı.")

        for annotation in data['annotations']:
            if annotation['image_id'] in image_id_mapping:
                ann_copy = annotation.copy()
                ann_copy['id'] = annotation['id'] + ann_id_offset
                ann_copy['image_id'] = image_id_mapping[annotation['image_id']]

                original_cat_name = next(cat['name'] for cat in data['categories'] if cat['id'] == annotation['category_id'])
                ann_copy['category_id'] = category_id_map[original_cat_name]

                combined_annotations.append(ann_copy)

        image_id_offset += processed_count
        ann_id_offset += len(data['annotations'])

    combined_data = {
        "info": {"description": "Combined COCO dataset"},
        "licenses": [],
        "images": combined_images,
        "annotations": combined_annotations,
        "categories": combined_categories
    }

    with open(output_ann_path, "w") as f:
        json.dump(combined_data, f, indent=4)

    print("\nVeri birleştirme tamamlandı.")

# --- Birleştirme İşlemini Başlat ---
paths_to_combine = [
    (annotations_path_1, source_dir_1),
    (annotations_path_2, source_dir_2)
]
combine_coco_datasets(paths_to_combine, combined_annotations_path, combined_dataset_dir)

# === 3. Analiz için Sınıf İsimleri ve Metadata ===
class_names = [
    "defect", "1- patlak", "2- igne_kirigi", "3- jut", "5- likra_kacigi", "6- yag_lekesi", "8- May cizgisi", "fsa"
]

unique_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
dataset_name = f"tekstil_combined_dataset_{unique_suffix}"

try:
    register_coco_instances(dataset_name, {}, combined_annotations_path, combined_dataset_dir)
except ValueError:
    pass

MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# === 4. Model Yükleme ===
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("\nModel loaded with Detectron2 DefaultPredictor.")

# === 5. Tahmin ve Karşılaştırma ===
gt_map = {}
filename_to_id = {}
all_image_paths = []

with open(combined_annotations_path, "r") as f:
    coco_ann = json.load(f)
    for img in coco_ann["images"]:
        filename = img["file_name"]
        img_id = img["id"]
        full_path = os.path.join(combined_dataset_dir, filename)
        filename_to_id[filename] = img_id
        all_image_paths.append(full_path)
    for ann in coco_ann["annotations"]:
        gt_map.setdefault(ann["image_id"], []).append(ann["category_id"])

y_true, y_pred = [], []
print(f"\nAnalyzing {len(all_image_paths)} images...")
for image_path in tqdm(all_image_paths):
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(os.path.basename(image_path))
    gt_classes = gt_map.get(image_id, [])

    for i in range(len(gt_classes)):
        y_true.append(gt_classes[i])
        if i < len(pred_classes):
            y_pred.append(pred_classes[i] + 1)
        else:
            y_pred.append(-1)

    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2)
        y_pred.append(pred_classes[i] + 1)

    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]
    save_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(save_path, out_img)

# === 6. Karmaşıklık Matrisi ve Detaylı Metrik Raporu ===
# Etiketleri, 1'den başlayan sınıf ID'leri ve -1 (None) ile oluştur
labels = list(range(1, len(class_names) + 1)) + [-1]
disp_labels = class_names + ["None"]

cm = confusion_matrix(y_true, y_pred, labels=labels)

# Karmaşıklık Matrisi görselleştirmesini kaydet
fig, ax = plt.subplots(figsize=(12, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=disp_labels)
disp.plot(include_values=True, xticks_rotation=45, ax=ax, cmap="Blues")
plt.title("Combined Confusion Matrix")
plt.tight_layout()
cm_path = os.path.join(output_dir, "combined_confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

# Detaylı TP, TN, FP, FN değerlerini hesaplayıp raporla
total_samples = len(y_true)
report_content = f"### Detaylı Metrik Raporu\n\nToplam Analiz Edilen Örnek Sayısı: {total_samples}\n\n"

for i, class_label in enumerate(disp_labels):
    tp = cm[i, i]
    fp = np.sum(cm[:, i]) - tp
    fn = np.sum(cm[i, :]) - tp
    tn = np.sum(cm) - (tp + fp + fn)

    tp_percent = (tp / total_samples) * 100 if total_samples > 0 else 0
    tn_percent = (tn / total_samples) * 100 if total_samples > 0 else 0
    fp_percent = (fp / total_samples) * 100 if total_samples > 0 else 0
    fn_percent = (fn / total_samples) * 100 if total_samples > 0 else 0

    report_content += f"--- {class_label} ---\n"
    report_content += f"TP (Doğru Pozitif): {tp} ({tp_percent:.2f}%)\n"
    report_content += f"TN (Doğru Negatif): {tn} ({tn_percent:.2f}%)\n"
    report_content += f"FP (Yanlış Pozitif): {fp} ({fp_percent:.2f}%)\n"
    report_content += f"FN (Yanlış Negatif): {fn} ({fn_percent:.2f}%)\n\n"

# Raporu dosyaya kaydet
detailed_report_path = os.path.join(output_dir, "detailed_metrics_report.txt")
with open(detailed_report_path, "w") as f:
    f.write(report_content)

# Sınıflandırma raporunu (precision, recall, f1-score) kaydet
report = classification_report(
    y_true, y_pred, labels=labels,
    target_names=disp_labels,
    zero_division=0
)
report_path = os.path.join(output_dir, "combined_classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)

print(f"\nAnaliz tamamlandı. Raporlar ve görseller buraya kaydedildi: {output_dir}")

In [ ]:
import os
import shutil
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch
import random
import string
import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

# Google Drive'ı bağlama (Colab için)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

# === 1. Dosya Yolları ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"

# Kaynak veri setlerinin yolları
source_dir_1 = "/#your file path/train"
annotations_path_1 = os.path.join(source_dir_1, "ikinci.coco.json")
source_dir_2 = "/#your file path/test"
annotations_path_2 = os.path.join(source_dir_2, "_annotations.coco.json")

# Birleştirilmiş verilerin ve çıktıların kaydedileceği ana klasörler
combined_dataset_dir = "/#your file path/combined_dataset"
combined_annotations_path = os.path.join(combined_dataset_dir, "combined_annotations.coco.json")
output_dir = "/#your file path/combined_analysis_report22"

# Klasörleri oluştur/temizle
if os.path.exists(combined_dataset_dir):
    shutil.rmtree(combined_dataset_dir)
os.makedirs(combined_dataset_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# === 2. Veri Kümelerini Birleştirme Fonksiyonu ===
def combine_coco_datasets(paths, output_ann_path, output_img_dir):
    combined_images = []
    combined_annotations = []
    combined_categories = []

    image_id_offset = 0
    ann_id_offset = 0
    category_id_map = {}

    print("Veri setleri birleştiriliyor...")

    for ann_path, img_dir in paths:
        if not os.path.exists(ann_path):
            print(f"Uyarı: Annotation dosyası bulunamadı, atlanıyor: {ann_path}")
            continue

        with open(ann_path, 'r') as f:
            data = json.load(f)

        for cat in data['categories']:
            if cat['name'] not in category_id_map:
                new_id = len(category_id_map) + 1
                category_id_map[cat['name']] = new_id
                combined_categories.append({'id': new_id, 'name': cat['name'], 'supercategory': cat.get('supercategory', 'none')})

        image_id_mapping = {}
        processed_count = 0
        skipped_count = 0

        for image in tqdm(data['images'], desc=f"Görseller kopyalanıyor: {img_dir}"):
            src_image_path = os.path.join(img_dir, image['file_name'])

            if os.path.exists(src_image_path):
                old_image_id = image['id']
                new_image_id = old_image_id + image_id_offset
                image_id_mapping[old_image_id] = new_image_id

                image_copy = image.copy()
                image_copy['id'] = new_image_id

                dst_image_path = os.path.join(output_img_dir, image['file_name'])
                shutil.copy(src_image_path, dst_image_path)

                combined_images.append(image_copy)
                processed_count += 1
            else:
                skipped_count += 1
                print(f"\nUyarı: {src_image_path} bulunamadı, bu görsel atlanıyor.")

        print(f"\n{img_dir} klasöründen {processed_count} görsel başarıyla kopyalandı, {skipped_count} görsel atlandı.")

        for annotation in data['annotations']:
            if annotation['image_id'] in image_id_mapping:
                ann_copy = annotation.copy()
                ann_copy['id'] = annotation['id'] + ann_id_offset
                ann_copy['image_id'] = image_id_mapping[annotation['image_id']]

                original_cat_name = next(cat['name'] for cat in data['categories'] if cat['id'] == annotation['category_id'])
                ann_copy['category_id'] = category_id_map[original_cat_name]

                combined_annotations.append(ann_copy)

        image_id_offset += processed_count
        ann_id_offset += len(data['annotations'])

    combined_data = {
        "info": {"description": "Combined COCO dataset"},
        "licenses": [],
        "images": combined_images,
        "annotations": combined_annotations,
        "categories": combined_categories
    }

    with open(output_ann_path, "w") as f:
        json.dump(combined_data, f, indent=4)

    print("\nVeri birleştirme tamamlandı.")

# --- Birleştirme İşlemini Başlat ---
paths_to_combine = [
    (annotations_path_1, source_dir_1),
    (annotations_path_2, source_dir_2)
]
combine_coco_datasets(paths_to_combine, combined_annotations_path, combined_dataset_dir)


# === 3. Analiz için Sınıf İsimleri ve Metadata ===
class_names = [
    "defect", "1- patlak", "2- igne_kirigi", "3- jut", "5- likra_kacigi", "6- yag_lekesi", "8- May cizgisi", "fsa"
]

unique_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
dataset_name = f"tekstil_combined_dataset_{unique_suffix}"

try:
    register_coco_instances(dataset_name, {}, combined_annotations_path, combined_dataset_dir)
except ValueError:
    pass

MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# === 4. Model Yükleme ===
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("\nModel loaded with Detectron2 DefaultPredictor.")

# === 5. Tahmin ve Karşılaştırma ===
gt_map = {}
filename_to_id = {}
all_image_paths = []

with open(combined_annotations_path, "r") as f:
    coco_ann = json.load(f)
    for img in coco_ann["images"]:
        filename = img["file_name"]
        img_id = img["id"]
        full_path = os.path.join(combined_dataset_dir, filename)
        filename_to_id[filename] = img_id
        all_image_paths.append(full_path)
    for ann in coco_ann["annotations"]:
        gt_map.setdefault(ann["image_id"], []).append(ann["category_id"])

y_true, y_pred = [], []
print(f"\nAnalyzing {len(all_image_paths)} images...")
for image_path in tqdm(all_image_paths):
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(os.path.basename(image_path))
    gt_classes = gt_map.get(image_id, [])

    for i in range(len(gt_classes)):
        y_true.append(gt_classes[i])
        if i < len(pred_classes):
            y_pred.append(pred_classes[i] + 1)
        else:
            y_pred.append(-1)

    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2)
        y_pred.append(pred_classes[i] + 1)

    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]
    save_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(save_path, out_img)

# === 6. Karmaşıklık Matrisi ve Detaylı Metrik Raporu ===
# Etiketleri, 1'den başlayan sınıf ID'leri ve -1 (None) ile oluştur
labels = list(range(1, len(class_names) + 1)) + [-1]
disp_labels = class_names + ["None"]

cm = confusion_matrix(y_true, y_pred, labels=labels)

# Karmaşıklık Matrisi görselleştirmesini kaydet
fig, ax = plt.subplots(figsize=(12, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=disp_labels)
disp.plot(include_values=True, xticks_rotation=45, ax=ax, cmap="Blues")
plt.title("Combined Confusion Matrix")
plt.tight_layout()
cm_path = os.path.join(output_dir, "combined_confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

# Detaylı TP, TN, FP, FN değerlerini hesaplayıp görselleştirme
total_samples = len(y_true)
num_classes = len(disp_labels)
metrics_data = np.zeros((num_classes, 4), dtype=int)
metrics_percentage_data = np.zeros((num_classes, 4), dtype=float)

for i in range(num_classes):
    tp = cm[i, i]
    fp = np.sum(cm[:, i]) - tp
    fn = np.sum(cm[i, :]) - tp
    tn = np.sum(cm) - (tp + fp + fn)

    metrics_data[i, 0] = tp
    metrics_data[i, 1] = fp
    metrics_data[i, 2] = fn
    metrics_data[i, 3] = tn

    tp_perc = (tp / total_samples) * 100 if total_samples > 0 else 0
    fp_perc = (fp / total_samples) * 100 if total_samples > 0 else 0
    fn_perc = (fn / total_samples) * 100 if total_samples > 0 else 0
    tn_perc = (tn / total_samples) * 100 if total_samples > 0 else 0

    metrics_percentage_data[i, 0] = tp_perc
    metrics_percentage_data[i, 1] = fp_perc
    metrics_percentage_data[i, 2] = fn_perc
    metrics_percentage_data[i, 3] = tn_perc

# Matrisi çizme
fig, ax = plt.subplots(figsize=(14, 8))
columns = ['TP (Doğru Pozitif)', 'FP (Yanlış Pozitif)', 'FN (Yanlış Negatif)', 'TN (Doğru Negatif)']
ax.set_title("Detaylı Metrik Matrisi")
ax.axis('tight')
ax.axis('off')

cell_text = []
for i in range(num_classes):
    row_text = []
    for j in range(4):
        value = metrics_data[i, j]
        percentage = metrics_percentage_data[i, j]
        row_text.append(f"{value}\n({percentage:.2f}%)")
    cell_text.append(row_text)

table = ax.table(cellText=cell_text, colLabels=columns, rowLabels=disp_labels, loc='center', cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)

for (i, j), cell in table.get_celld().items():
    if i == 0:  # Başlık satırı
        cell.set_facecolor("#4CAF50")
    else:
        # TP ve TN için yeşil, FP ve FN için kırmızı
        if j == 0 or j == 3:
            cell.set_facecolor("#E8F5E9") # Açık yeşil
        else:
            cell.set_facecolor("#FFEBEE") # Açık kırmızı

plt.tight_layout()
metrics_matrix_path = os.path.join(output_dir, "detailed_metrics_matrix.png")
plt.savefig(metrics_matrix_path, bbox_inches='tight', pad_inches=0.5)
plt.close()

# Sınıflandırma raporunu (precision, recall, f1-score) kaydet
report = classification_report(
    y_true, y_pred, labels=labels,
    target_names=disp_labels,
    zero_division=0
)
report_path = os.path.join(output_dir, "combined_classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)

print(f"\nAnaliz tamamlandı. Raporlar ve görseller buraya kaydedildi: {output_dir}")

In [ ]:
import os
import shutil
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch
import random
import string
import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from matplotlib.colors import LinearSegmentedColormap

# Google Drive'ı bağlama (Colab için)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

# === 1. Dosya Yolları ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"

# Kaynak veri setlerinin yolları
source_dir_1 = "/#your file path/train"
annotations_path_1 = os.path.join(source_dir_1, "ikinci.coco.json")
source_dir_2 = "/#your file path/test"
annotations_path_2 = os.path.join(source_dir_2, "_annotations.coco.json")

# Birleştirilmiş verilerin ve çıktıların kaydedileceği ana klasörler
combined_dataset_dir = "/#your file path/combined_dataset"
combined_annotations_path = os.path.join(combined_dataset_dir, "combined_annotations.coco.json")
output_dir = "/#your file path/combined_analysis_report55"

# Klasörleri oluştur/temizle
if os.path.exists(combined_dataset_dir):
    shutil.rmtree(combined_dataset_dir)
os.makedirs(combined_dataset_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# === 2. Veri Kümelerini Birleştirme Fonksiyonu ===
def combine_coco_datasets(paths, output_ann_path, output_img_dir):
    combined_images = []
    combined_annotations = []
    combined_categories = []

    image_id_offset = 0
    ann_id_offset = 0
    category_id_map = {}

    print("Veri setleri birleştiriliyor...")

    for ann_path, img_dir in paths:
        if not os.path.exists(ann_path):
            print(f"Uyarı: Annotation dosyası bulunamadı, atlanıyor: {ann_path}")
            continue

        with open(ann_path, 'r') as f:
            data = json.load(f)

        for cat in data['categories']:
            if cat['name'] not in category_id_map:
                new_id = len(category_id_map) + 1
                category_id_map[cat['name']] = new_id
                combined_categories.append({'id': new_id, 'name': cat['name'], 'supercategory': cat.get('supercategory', 'none')})

        image_id_mapping = {}
        processed_count = 0
        skipped_count = 0

        for image in tqdm(data['images'], desc=f"Görseller kopyalanıyor: {img_dir}"):
            src_image_path = os.path.join(img_dir, image['file_name'])

            if os.path.exists(src_image_path):
                old_image_id = image['id']
                new_image_id = old_image_id + image_id_offset
                image_id_mapping[old_image_id] = new_image_id

                image_copy = image.copy()
                image_copy['id'] = new_image_id

                dst_image_path = os.path.join(output_img_dir, image['file_name'])
                shutil.copy(src_image_path, dst_image_path)

                combined_images.append(image_copy)
                processed_count += 1
            else:
                skipped_count += 1
                print(f"\nUyarı: {src_image_path} bulunamadı, bu görsel atlanıyor.")

        print(f"\n{img_dir} klasöründen {processed_count} görsel başarıyla kopyalandı, {skipped_count} görsel atlandı.")

        for annotation in data['annotations']:
            if annotation['image_id'] in image_id_mapping:
                ann_copy = annotation.copy()
                ann_copy['id'] = annotation['id'] + ann_id_offset
                ann_copy['image_id'] = image_id_mapping[annotation['image_id']]

                original_cat_name = next(cat['name'] for cat in data['categories'] if cat['id'] == annotation['category_id'])
                ann_copy['category_id'] = category_id_map[original_cat_name]

                combined_annotations.append(ann_copy)

        image_id_offset += processed_count
        ann_id_offset += len(data['annotations'])

    combined_data = {
        "info": {"description": "Combined COCO dataset"},
        "licenses": [],
        "images": combined_images,
        "annotations": combined_annotations,
        "categories": combined_categories
    }

    with open(output_ann_path, "w") as f:
        json.dump(combined_data, f, indent=4)

    print("\nVeri birleştirme tamamlandı.")

# --- Birleştirme İşlemini Başlat ---
paths_to_combine = [
    (annotations_path_1, source_dir_1),
    (annotations_path_2, source_dir_2)
]
combine_coco_datasets(paths_to_combine, combined_annotations_path, combined_dataset_dir)


# === 3. Analiz için Sınıf İsimleri ve Metadata ===
class_names = [
    "defect", "1- patlak", "2- igne_kirigi", "3- jut", "5- likra_kacigi", "6- yag_lekesi", "8- May cizgisi", "fsa"
]

unique_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
dataset_name = f"tekstil_combined_dataset_{unique_suffix}"

try:
    register_coco_instances(dataset_name, {}, combined_annotations_path, combined_dataset_dir)
except ValueError:
    pass

MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# === 4. Model Yükleme ===
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("\nModel loaded with Detectron2 DefaultPredictor.")


# === 5. Tahmin ve Karşılaştırma ===
gt_map = {}
filename_to_id = {}
all_image_paths = []

with open(combined_annotations_path, "r") as f:
    coco_ann = json.load(f)
    for img in coco_ann["images"]:
        filename = img["file_name"]
        img_id = img["id"]
        full_path = os.path.join(combined_dataset_dir, filename)
        filename_to_id[filename] = img_id
        all_image_paths.append(full_path)
    for ann in coco_ann["annotations"]:
        gt_map.setdefault(ann["image_id"], []).append(ann["category_id"])

y_true, y_pred = [], []
print(f"\nAnalyzing {len(all_image_paths)} images...")
for image_path in tqdm(all_image_paths):
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(os.path.basename(image_path))
    gt_classes = gt_map.get(image_id, [])

    for i in range(len(gt_classes)):
        y_true.append(gt_classes[i])
        if i < len(pred_classes):
            y_pred.append(pred_classes[i] + 1)
        else:
            y_pred.append(-1)

    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2)
        y_pred.append(pred_classes[i] + 1)

    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]
    save_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(save_path, out_img)

# === 6. Karmaşıklık Matrisi ve Detaylı Metrik Raporu ===
labels = list(range(1, len(class_names) + 1)) + [-1]
disp_labels = class_names + ["None"]

cm = confusion_matrix(y_true, y_pred, labels=labels)

# Karmaşıklık Matrisi görselleştirmesini kaydet
fig, ax = plt.subplots(figsize=(12, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=disp_labels)
disp.plot(include_values=True, xticks_rotation=45, ax=ax, cmap="Blues")
plt.title("Combined Confusion Matrix")
plt.tight_layout()
cm_path = os.path.join(output_dir, "combined_confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

# Detaylı TP, TN, FP, FN değerlerini hesaplayıp görselleştirme
total_samples = len(y_true)
num_classes = len(disp_labels)

metrics_data = np.zeros((num_classes, 4), dtype=int)
metrics_percentage_data = np.zeros((num_classes, 4), dtype=float)

for i in range(num_classes):
    tp = cm[i, i]
    fp = np.sum(cm[:, i]) - tp
    fn = np.sum(cm[i, :]) - tp
    tn = np.sum(cm) - (tp + fp + fn)

    metrics_data[i, 0] = tp
    metrics_data[i, 1] = fp
    metrics_data[i, 2] = fn
    metrics_data

In [ ]:
import os
import shutil
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch
import random
import string
import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

# Google Drive'ı bağlama (Colab için)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

# === 1. Dosya Yolları ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"

# Kaynak veri setlerinin yolları
source_dir_1 = "/#your file path/train"
annotations_path_1 = os.path.join(source_dir_1, "ikinci.coco.json")
source_dir_2 = "/#your file path/test"
annotations_path_2 = os.path.join(source_dir_2, "_annotations.coco.json")

# Birleştirilmiş verilerin ve çıktıların kaydedileceği ana klasörler
combined_dataset_dir = "/#your file path/combined_dataset"
combined_annotations_path = os.path.join(combined_dataset_dir, "combined_annotations.coco.json")
output_dir = "/#your file path/combined_analysis_report66"

# Klasörleri oluştur/temizle
if os.path.exists(combined_dataset_dir):
    shutil.rmtree(combined_dataset_dir)
os.makedirs(combined_dataset_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# === 2. Veri Kümelerini Birleştirme Fonksiyonu ===
def combine_coco_datasets(paths, output_ann_path, output_img_dir):
    combined_images = []
    combined_annotations = []
    combined_categories = []

    image_id_offset = 0
    ann_id_offset = 0
    category_id_map = {}

    print("Veri setleri birleştiriliyor...")

    for ann_path, img_dir in paths:
        if not os.path.exists(ann_path):
            print(f"Uyarı: Annotation dosyası bulunamadı, atlanıyor: {ann_path}")
            continue

        with open(ann_path, 'r') as f:
            data = json.load(f)

        for cat in data['categories']:
            if cat['name'] not in category_id_map:
                new_id = len(category_id_map) + 1
                category_id_map[cat['name']] = new_id
                combined_categories.append({'id': new_id, 'name': cat['name'], 'supercategory': cat.get('supercategory', 'none')})

        image_id_mapping = {}
        processed_count = 0
        skipped_count = 0

        for image in tqdm(data['images'], desc=f"Görseller kopyalanıyor: {img_dir}"):
            src_image_path = os.path.join(img_dir, image['file_name'])

            if os.path.exists(src_image_path):
                old_image_id = image['id']
                new_image_id = old_image_id + image_id_offset
                image_id_mapping[old_image_id] = new_image_id

                image_copy = image.copy()
                image_copy['id'] = new_image_id

                dst_image_path = os.path.join(output_img_dir, image['file_name'])
                shutil.copy(src_image_path, dst_image_path)

                combined_images.append(image_copy)
                processed_count += 1
            else:
                skipped_count += 1
                print(f"\nUyarı: {src_image_path} bulunamadı, bu görsel atlanıyor.")

        print(f"\n{img_dir} klasöründen {processed_count} görsel başarıyla kopyalandı, {skipped_count} görsel atlandı.")

        for annotation in data['annotations']:
            if annotation['image_id'] in image_id_mapping:
                ann_copy = annotation.copy()
                ann_copy['id'] = annotation['id'] + ann_id_offset
                ann_copy['image_id'] = image_id_mapping[annotation['image_id']]

                original_cat_name = next(cat['name'] for cat in data['categories'] if cat['id'] == annotation['category_id'])
                ann_copy['category_id'] = category_id_map[original_cat_name]

                combined_annotations.append(ann_copy)

        image_id_offset += processed_count
        ann_id_offset += len(data['annotations'])

    combined_data = {
        "info": {"description": "Combined COCO dataset"},
        "licenses": [],
        "images": combined_images,
        "annotations": combined_annotations,
        "categories": combined_categories
    }

    with open(output_ann_path, "w") as f:
        json.dump(combined_data, f, indent=4)

    print("\nVeri birleştirme tamamlandı.")

# --- Birleştirme İşlemini Başlat ---
paths_to_combine = [
    (annotations_path_1, source_dir_1),
    (annotations_path_2, source_dir_2)
]
combine_coco_datasets(paths_to_combine, combined_annotations_path, combined_dataset_dir)


# === 3. Analiz için Sınıf İsimleri ve Metadata ===
class_names = [
    "defect", "1- patlak", "2- igne_kirigi", "3- jut", "5- likra_kacigi", "6- yag_lekesi", "8- May cizgisi", "fsa"
]

unique_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
dataset_name = f"tekstil_combined_dataset_{unique_suffix}"

try:
    register_coco_instances(dataset_name, {}, combined_annotations_path, combined_dataset_dir)
except ValueError:
    pass

MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# === 4. Model Yükleme ===
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("\nModel loaded with Detectron2 DefaultPredictor.")


# === 5. Tahmin ve Karşılaştırma ===
gt_map = {}
filename_to_id = {}
all_image_paths = []

with open(combined_annotations_path, "r") as f:
    coco_ann = json.load(f)
    for img in coco_ann["images"]:
        filename = img["file_name"]
        img_id = img["id"]
        full_path = os.path.join(combined_dataset_dir, filename)
        filename_to_id[filename] = img_id
        all_image_paths.append(full_path)
    for ann in coco_ann["annotations"]:
        gt_map.setdefault(ann["image_id"], []).append(ann["category_id"])

y_true, y_pred = [], []
print(f"\nAnalyzing {len(all_image_paths)} images...")
for image_path in tqdm(all_image_paths):
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(os.path.basename(image_path))
    gt_classes = gt_map.get(image_id, [])

    for i in range(len(gt_classes)):
        y_true.append(gt_classes[i])
        if i < len(pred_classes):
            y_pred.append(pred_classes[i] + 1)
        else:
            y_pred.append(-1)

    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2)
        y_pred.append(pred_classes[i] + 1)

    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]
    save_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(save_path, out_img)

# === 6. Genel Başarı Matrisi (TP/FP/FN/TN) Oluşturma ===
# Tüm defect sınıflarını "Hatalı" olarak, None ve Diğer sınıfları "Hatasız" olarak kabul etme
defect_class_ids = set(range(1, len(class_names) + 1))
y_true_binary = [1 if val in defect_class_ids else 0 for val in y_true]
y_pred_binary = [1 if val in defect_class_ids else 0 for val in y_pred]

# 2x2 karışıklık matrisini hesapla
cm = confusion_matrix(y_true_binary, y_pred_binary, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()
total_samples = tn + fp + fn + tp

# Matrisi istenen düzende görselleştirme
fig, ax = plt.subplots(figsize=(8, 8))
ax.axis('off')
ax.set_title('Genel Başarı Matrisi', fontsize=18, pad=20)

# Verileri ve etiketleri hazırlama
cell_texts = np.array([
    [f"TP: {tp}\n({tp/total_samples*100:.2f}%)", f"FP: {fp}\n({fp/total_samples*100:.2f}%)"],
    [f"FN: {fn}\n({fn/total_samples*100:.2f}%)", f"TN: {tn}\n({tn/total_samples*100:.2f}%)"]
])
row_labels = ["Gerçek Hatalı (Expected)", "Gerçek Hatasız"]
col_labels = ["Tahmin Edilen Hatalı (Predicted)", "Tahmin Edilen Hatasız"]

# Matrisi çizme
table = ax.table(cellText=cell_texts,
                 rowLabels=row_labels,
                 colLabels=col_labels,
                 loc='center',
                 cellLoc='center')

table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1.2, 1.2)

# Renklendirme
table.get_celld()[1, 0].set_facecolor('#d9f1e1')  # TP
table.get_celld()[1, 1].set_facecolor('#f8e2e2')  # FP
table.get_celld()[0, 0].set_facecolor('#f8e2e2')  # FN
table.get_celld()[0, 1].set_facecolor('#d9f1e1')  # TN

plt.tight_layout()
matrix_path = os.path.join(output_dir, "genel_basari_matrisi.png")
plt.savefig(matrix_path, bbox_inches='tight', pad_inches=0.5)
plt.close()

# Sınıflandırma raporunu (precision, recall, f1-score) kaydet
report = classification_report(
    y_true, y_pred, labels=labels,
    target_names=disp_labels,
    zero_division=0
)
report_path = os.path.join(output_dir, "combined_classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)

print(f"\nAnaliz tamamlandı. Genel Başarı Matrisi buraya kaydedildi: {output_dir}")

In [ ]:
import os
import shutil
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch
import random
import string
import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

# Google Drive'ı bağlama (Colab için)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

# === 1. Dosya Yolları ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"

# Kaynak veri setlerinin yolları
source_dir_1 = "/#your file path/train"
annotations_path_1 = os.path.join(source_dir_1, "ikinci.coco.json")
source_dir_2 = "/#your file path/test"
annotations_path_2 = os.path.join(source_dir_2, "_annotations.coco.json")

# Birleştirilmiş verilerin ve çıktıların kaydedileceği ana klasörler
combined_dataset_dir = "/#your file path/combined_dataset"
combined_annotations_path = os.path.join(combined_dataset_dir, "combined_annotations.coco.json")
output_dir = "/#your file path/combined_analysis_report666"

# Klasörleri oluştur/temizle
if os.path.exists(combined_dataset_dir):
    shutil.rmtree(combined_dataset_dir)
os.makedirs(combined_dataset_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# === 2. Veri Kümelerini Birleştirme Fonksiyonu ===
def combine_coco_datasets(paths, output_ann_path, output_img_dir):
    combined_images = []
    combined_annotations = []
    combined_categories = []

    image_id_offset = 0
    ann_id_offset = 0
    category_id_map = {}

    print("Veri setleri birleştiriliyor...")

    for ann_path, img_dir in paths:
        if not os.path.exists(ann_path):
            print(f"Uyarı: Annotation dosyası bulunamadı, atlanıyor: {ann_path}")
            continue

        with open(ann_path, 'r') as f:
            data = json.load(f)

        for cat in data['categories']:
            if cat['name'] not in category_id_map:
                new_id = len(category_id_map) + 1
                category_id_map[cat['name']] = new_id
                combined_categories.append({'id': new_id, 'name': cat['name'], 'supercategory': cat.get('supercategory', 'none')})

        image_id_mapping = {}
        processed_count = 0
        skipped_count = 0

        for image in tqdm(data['images'], desc=f"Görseller kopyalanıyor: {img_dir}"):
            src_image_path = os.path.join(img_dir, image['file_name'])

            if os.path.exists(src_image_path):
                old_image_id = image['id']
                new_image_id = old_image_id + image_id_offset
                image_id_mapping[old_image_id] = new_image_id

                image_copy = image.copy()
                image_copy['id'] = new_image_id

                dst_image_path = os.path.join(output_img_dir, image['file_name'])
                shutil.copy(src_image_path, dst_image_path)

                combined_images.append(image_copy)
                processed_count += 1
            else:
                skipped_count += 1
                print(f"\nUyarı: {src_image_path} bulunamadı, bu görsel atlanıyor.")

        print(f"\n{img_dir} klasöründen {processed_count} görsel başarıyla kopyalandı, {skipped_count} görsel atlandı.")

        for annotation in data['annotations']:
            if annotation['image_id'] in image_id_mapping:
                ann_copy = annotation.copy()
                ann_copy['id'] = annotation['id'] + ann_id_offset
                ann_copy['image_id'] = image_id_mapping[annotation['image_id']]

                original_cat_name = next(cat['name'] for cat in data['categories'] if cat['id'] == annotation['category_id'])
                ann_copy['category_id'] = category_id_map[original_cat_name]

                combined_annotations.append(ann_copy)

        image_id_offset += processed_count
        ann_id_offset += len(data['annotations'])

    combined_data = {
        "info": {"description": "Combined COCO dataset"},
        "licenses": [],
        "images": combined_images,
        "annotations": combined_annotations,
        "categories": combined_categories
    }

    with open(output_ann_path, "w") as f:
        json.dump(combined_data, f, indent=4)

    print("\nVeri birleştirme tamamlandı.")

# --- Birleştirme İşlemini Başlat ---
paths_to_combine = [
    (annotations_path_1, source_dir_1),
    (annotations_path_2, source_dir_2)
]
combine_coco_datasets(paths_to_combine, combined_annotations_path, combined_dataset_dir)


# === 3. Analiz için Sınıf İsimleri ve Metadata ===
class_names = [
    "defect", "1- patlak", "2- igne_kirigi", "3- jut", "5- likra_kacigi", "6- yag_lekesi", "8- May cizgisi", "fsa"
]

unique_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
dataset_name = f"tekstil_combined_dataset_{unique_suffix}"

try:
    register_coco_instances(dataset_name, {}, combined_annotations_path, combined_dataset_dir)
except ValueError:
    pass

MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# === 4. Model Yükleme ===
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("\nModel loaded with Detectron2 DefaultPredictor.")


# === 5. Tahmin ve Karşılaştırma ===
gt_map = {}
filename_to_id = {}
all_image_paths = []

with open(combined_annotations_path, "r") as f:
    coco_ann = json.load(f)
    for img in coco_ann["images"]:
        filename = img["file_name"]
        img_id = img["id"]
        full_path = os.path.join(combined_dataset_dir, filename)
        filename_to_id[filename] = img_id
        all_image_paths.append(full_path)
    for ann in coco_ann["annotations"]:
        gt_map.setdefault(ann["image_id"], []).append(ann["category_id"])

y_true, y_pred = [], []
print(f"\nAnalyzing {len(all_image_paths)} images...")
for image_path in tqdm(all_image_paths):
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(os.path.basename(image_path))
    gt_classes = gt_map.get(image_id, [])

    for i in range(len(gt_classes)):
        y_true.append(gt_classes[i])
        if i < len(pred_classes):
            y_pred.append(pred_classes[i] + 1)
        else:
            y_pred.append(-1)

    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2)
        y_pred.append(pred_classes[i] + 1)

    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]
    save_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(save_path, out_img)

# === 6. Genel Başarı Matrisi (TP/FP/FN/TN) Oluşturma ===
# Tüm defect sınıflarını "Hatalı" olarak, None ve Diğer sınıfları "Hatasız" olarak kabul etme
defect_class_ids = set(range(1, len(class_names) + 1))
y_true_binary = [1 if val in defect_class_ids else 0 for val in y_true]
y_pred_binary = [1 if val in defect_class_ids else 0 for val in y_pred]

# 2x2 karışıklık matrisini hesapla
cm = confusion_matrix(y_true_binary, y_pred_binary, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()
total_samples = tn + fp + fn + tp

# Matrisi istenen düzende görselleştirme
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_title('Genel Başarı Matrisi', fontsize=18, pad=20)
ax.set_xlabel('Tahmin Edilen', fontsize=14)
ax.set_ylabel('Gerçek', fontsize=14)

# Renk haritası için veri
heatmap_data = np.array([[tp, fp], [fn, tn]])
normalized_data = heatmap_data / np.max(heatmap_data) if np.max(heatmap_data) > 0 else np.zeros((2,2))

# Renklendirme
cmap = plt.cm.get_cmap('Greens')
cmap_r = plt.cm.get_cmap('Reds')

# TP ve TN için yeşil tonları, FP ve FN için kırmızı tonları
colors = np.zeros((2, 2, 4))
colors[0, 0] = cmap(normalized_data[0, 0]) # TP
colors[0, 1] = cmap_r(normalized_data[0, 1]) # FP
colors[1, 0] = cmap_r(normalized_data[1, 0]) # FN
colors[1, 1] = cmap(normalized_data[1, 1]) # TN

# Matrisin kendisi
ax.imshow(colors, interpolation='nearest', origin='upper')

# Metinleri hücrelere yazma
labels_text = [
    [f"TP\n{tp}\n({tp/total_samples*100:.2f}%)", f"FP\n{fp}\n({fp/total_samples*100:.2f}%)"],
    [f"FN\n{fn}\n({fn/total_samples*100:.2f}%)", f"TN\n{tn}\n({tn/total_samples*100:.2f}%)"]
]
for i in range(2):
    for j in range(2):
        ax.text(j, i, labels_text[i][j], ha='center', va='center', color='black', fontsize=16, weight='bold')

# Eksen etiketlerini ve işaretlerini ayarla
ax.set_xticks(np.arange(2))
ax.set_yticks(np.arange(2))
ax.set_xticklabels(["Hatalı", "Hatasız"])
ax.set_yticklabels(["Hatalı", "Hatasız"])

plt.tight_layout()
matrix_path = os.path.join(output_dir, "genel_basari_matrisi.png")
plt.savefig(matrix_path, bbox_inches='tight', pad_inches=0.5)
plt.close()

# Sınıflandırma raporunu (precision, recall, f1-score) kaydet
report = classification_report(
    y_true, y_pred, labels=labels,
    target_names=disp_labels,
    zero_division=0
)
report_path = os.path.join(output_dir, "combined_classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)

print(f"\nAnaliz tamamlandı. Genel Başarı Matrisi buraya kaydedildi: {output_dir}")

In [ ]:
import os
import shutil
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch
import random
import string
import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

# Google Drive'ı bağlama (Colab için)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

# === 1. Dosya Yolları ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"

# Kaynak veri setlerinin yolları
source_dir_1 = "/#your file path/train"
annotations_path_1 = os.path.join(source_dir_1, "ikinci.coco.json")
source_dir_2 = "/#your file path/test"
annotations_path_2 = os.path.join(source_dir_2, "_annotations.coco.json")

# Birleştirilmiş verilerin ve çıktıların kaydedileceği ana klasörler
combined_dataset_dir = "/#your file path/combined_dataset"
combined_annotations_path = os.path.join(combined_dataset_dir, "combined_annotations.coco.json")
output_dir = "/#your file path/combined_analysis_report6666"

# Klasörleri oluştur/temizle
if os.path.exists(combined_dataset_dir):
    shutil.rmtree(combined_dataset_dir)
os.makedirs(combined_dataset_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# === 2. Veri Kümelerini Birleştirme Fonksiyonu ===
def combine_coco_datasets(paths, output_ann_path, output_img_dir):
    combined_images = []
    combined_annotations = []
    combined_categories = []

    image_id_offset = 0
    ann_id_offset = 0
    category_id_map = {}

    print("Veri setleri birleştiriliyor...")

    for ann_path, img_dir in paths:
        if not os.path.exists(ann_path):
            print(f"Uyarı: Annotation dosyası bulunamadı, atlanıyor: {ann_path}")
            continue

        with open(ann_path, 'r') as f:
            data = json.load(f)

        for cat in data['categories']:
            if cat['name'] not in category_id_map:
                new_id = len(category_id_map) + 1
                category_id_map[cat['name']] = new_id
                combined_categories.append({'id': new_id, 'name': cat['name'], 'supercategory': cat.get('supercategory', 'none')})

        image_id_mapping = {}
        processed_count = 0
        skipped_count = 0

        for image in tqdm(data['images'], desc=f"Görseller kopyalanıyor: {img_dir}"):
            src_image_path = os.path.join(img_dir, image['file_name'])

            if os.path.exists(src_image_path):
                old_image_id = image['id']
                new_image_id = old_image_id + image_id_offset
                image_id_mapping[old_image_id] = new_image_id

                image_copy = image.copy()
                image_copy['id'] = new_image_id

                dst_image_path = os.path.join(output_img_dir, image['file_name'])
                shutil.copy(src_image_path, dst_image_path)

                combined_images.append(image_copy)
                processed_count += 1
            else:
                skipped_count += 1
                print(f"\nUyarı: {src_image_path} bulunamadı, bu görsel atlanıyor.")

        print(f"\n{img_dir} klasöründen {processed_count} görsel başarıyla kopyalandı, {skipped_count} görsel atlandı.")

        for annotation in data['annotations']:
            if annotation['image_id'] in image_id_mapping:
                ann_copy = annotation.copy()
                ann_copy['id'] = annotation['id'] + ann_id_offset
                ann_copy['image_id'] = image_id_mapping[annotation['image_id']]

                original_cat_name = next(cat['name'] for cat in data['categories'] if cat['id'] == annotation['category_id'])
                ann_copy['category_id'] = category_id_map[original_cat_name]

                combined_annotations.append(ann_copy)

        image_id_offset += processed_count
        ann_id_offset += len(data['annotations'])

    combined_data = {
        "info": {"description": "Combined COCO dataset"},
        "licenses": [],
        "images": combined_images,
        "annotations": combined_annotations,
        "categories": combined_categories
    }

    with open(output_ann_path, "w") as f:
        json.dump(combined_data, f, indent=4)

    print("\nVeri birleştirme tamamlandı.")

# --- Birleştirme İşlemini Başlat ---
paths_to_combine = [
    (annotations_path_1, source_dir_1),
    (annotations_path_2, source_dir_2)
]
combine_coco_datasets(paths_to_combine, combined_annotations_path, combined_dataset_dir)


# === 3. Analiz için Sınıf İsimleri ve Metadata ===
class_names = [
    "defect", "1- patlak", "2- igne_kirigi", "3- jut", "5- likra_kacigi", "6- yag_lekesi", "8- May cizgisi", "fsa"
]

unique_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
dataset_name = f"tekstil_combined_dataset_{unique_suffix}"

try:
    register_coco_instances(dataset_name, {}, combined_annotations_path, combined_dataset_dir)
except ValueError:
    pass

MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# === 4. Model Yükleme ===
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("\nModel loaded with Detectron2 DefaultPredictor.")


# === 5. Tahmin ve Karşılaştırma ===
gt_map = {}
filename_to_id = {}
all_image_paths = []

with open(combined_annotations_path, "r") as f:
    coco_ann = json.load(f)
    for img in coco_ann["images"]:
        filename = img["file_name"]
        img_id = img["id"]
        full_path = os.path.join(combined_dataset_dir, filename)
        filename_to_id[filename] = img_id
        all_image_paths.append(full_path)
    for ann in coco_ann["annotations"]:
        gt_map.setdefault(ann["image_id"], []).append(ann["category_id"])

y_true, y_pred = [], []
print(f"\nAnalyzing {len(all_image_paths)} images...")
for image_path in tqdm(all_image_paths):
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(os.path.basename(image_path))
    gt_classes = gt_map.get(image_id, [])

    for i in range(len(gt_classes)):
        y_true.append(gt_classes[i])
        if i < len(pred_classes):
            y_pred.append(pred_classes[i] + 1)
        else:
            y_pred.append(-1)

    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2)
        y_pred.append(pred_classes[i] + 1)

    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]
    save_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(save_path, out_img)

# === 6. Genel Başarı Matrisi (TP/FP/FN/TN) Oluşturma ===

# Tüm defect sınıflarını "Hatalı" olarak, None ve Diğer sınıfları "Hatasız" olarak kabul etme
defect_class_ids = set(range(1, len(class_names) + 1))
y_true_binary = [1 if val in defect_class_ids else 0 for val in y_true]
y_pred_binary = [1 if val in defect_class_ids else 0 for val in y_pred]

# 2x2 karışıklık matrisini hesapla
cm = confusion_matrix(y_true_binary, y_pred_binary, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()
total_samples = tn + fp + fn + tp

# Matrisi istenen düzende görselleştirme
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_title('Genel Başarı Matrisi', fontsize=18, pad=20)
ax.set_xlabel('Tahmin Edilen', fontsize=14)
ax.set_ylabel('Gerçek', fontsize=14)

# Renk haritası için veri
heatmap_data = np.array([[tp, fp], [fn, tn]])
normalized_data = heatmap_data / np.max(heatmap_data) if np.max(heatmap_data) > 0 else np.zeros((2,2))

# Renklendirme
cmap_green = plt.cm.get_cmap('Greens')
cmap_red = plt.cm.get_cmap('Reds')

# TP ve TN için yeşil tonları, FP ve FN için kırmızı tonları
colors = np.zeros((2, 2, 4))
colors[0, 0] = cmap_green(normalized_data[0, 0]) # TP
colors[0, 1] = cmap_red(normalized_data[0, 1])   # FP
colors[1, 0] = cmap_red(normalized_data[1, 0])   # FN
colors[1, 1] = cmap_green(normalized_data[1, 1]) # TN

# Matrisin kendisi
ax.imshow(colors, interpolation='nearest', origin='upper')

# Metinleri hücrelere yazma
labels_text = [
    [f"TP\n{tp}\n({tp/total_samples*100:.2f}%)", f"FP\n{fp}\n({fp/total_samples*100:.2f}%)"],
    [f"FN\n{fn}\n({fn/total_samples*100:.2f}%)", f"TN\n{tn}\n({tn/total_samples*100:.2f}%)"]
]
for i in range(2):
    for j in range(2):
        ax.text(j, i, labels_text[i][j], ha='center', va='center', color='black', fontsize=16, weight='bold')

# Eksen etiketlerini ve işaretlerini ayarla
ax.set_xticks(np.arange(2))
ax.set_yticks(np.arange(2))
ax.set_xticklabels(["Hatalı", "Hatasız"])
ax.set_yticklabels(["Hatalı", "Hatasız"])

plt.tight_layout()
matrix_path = os.path.join(output_dir, "genel_basari_matrisi.png")
plt.savefig(matrix_path, bbox_inches='tight', pad_inches=0.5)
plt.close()

# Sınıflandırma raporunu (precision, recall, f1-score) kaydet
report = classification_report(
    y_true_binary, y_pred_binary, labels=[0, 1],
    target_names=["Hatasız", "Hatalı"],
    zero_division=0
)
report_path = os.path.join(output_dir, "binary_classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)

print(f"\nAnaliz tamamlandı. Genel Başarı Matrisi buraya kaydedildi: {output_dir}")

In [ ]:
import os
import shutil
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch
import random
import string
import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

# Google Drive'ı bağlama (Colab için)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

# === 1. Dosya Yolları ===
config_path = "/#your file path/config.yaml"
weights_path = "/#your file path/model_final.pth"

# Kaynak veri setlerinin yolları
source_dir_1 = "/#your file path/train"
annotations_path_1 = os.path.join(source_dir_1, "ikinci.coco.json")
source_dir_2 = "/#your file path/test"
annotations_path_2 = os.path.join(source_dir_2, "_annotations.coco.json")

# Birleştirilmiş verilerin ve çıktıların kaydedileceği ana klasörler
combined_dataset_dir = "/#your file path/combined_dataset"
combined_annotations_path = os.path.join(combined_dataset_dir, "combined_annotations.coco.json")
output_dir = "/#your file path/combined_analysis_report66666"

# Klasörleri oluştur/temizle
if os.path.exists(combined_dataset_dir):
    shutil.rmtree(combined_dataset_dir)
os.makedirs(combined_dataset_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# === 2. Veri Kümelerini Birleştirme Fonksiyonu ===
def combine_coco_datasets(paths, output_ann_path, output_img_dir):
    combined_images = []
    combined_annotations = []
    combined_categories = []

    image_id_offset = 0
    ann_id_offset = 0
    category_id_map = {}

    print("Veri setleri birleştiriliyor...")

    for ann_path, img_dir in paths:
        if not os.path.exists(ann_path):
            print(f"Uyarı: Annotation dosyası bulunamadı, atlanıyor: {ann_path}")
            continue

        with open(ann_path, 'r') as f:
            data = json.load(f)

        for cat in data['categories']:
            if cat['name'] not in category_id_map:
                new_id = len(category_id_map) + 1
                category_id_map[cat['name']] = new_id
                combined_categories.append({'id': new_id, 'name': cat['name'], 'supercategory': cat.get('supercategory', 'none')})

        image_id_mapping = {}
        processed_count = 0
        skipped_count = 0

        for image in tqdm(data['images'], desc=f"Görseller kopyalanıyor: {img_dir}"):
            src_image_path = os.path.join(img_dir, image['file_name'])

            if os.path.exists(src_image_path):
                old_image_id = image['id']
                new_image_id = old_image_id + image_id_offset
                image_id_mapping[old_image_id] = new_image_id

                image_copy = image.copy()
                image_copy['id'] = new_image_id

                dst_image_path = os.path.join(output_img_dir, image['file_name'])
                shutil.copy(src_image_path, dst_image_path)

                combined_images.append(image_copy)
                processed_count += 1
            else:
                skipped_count += 1
                print(f"\nUyarı: {src_image_path} bulunamadı, bu görsel atlanıyor.")

        print(f"\n{img_dir} klasöründen {processed_count} görsel başarıyla kopyalandı, {skipped_count} görsel atlandı.")

        for annotation in data['annotations']:
            if annotation['image_id'] in image_id_mapping:
                ann_copy = annotation.copy()
                ann_copy['id'] = annotation['id'] + ann_id_offset
                ann_copy['image_id'] = image_id_mapping[annotation['image_id']]

                original_cat_name = next(cat['name'] for cat in data['categories'] if cat['id'] == annotation['category_id'])
                ann_copy['category_id'] = category_id_map[original_cat_name]

                combined_annotations.append(ann_copy)

        image_id_offset += processed_count
        ann_id_offset += len(data['annotations'])

    combined_data = {
        "info": {"description": "Combined COCO dataset"},
        "licenses": [],
        "images": combined_images,
        "annotations": combined_annotations,
        "categories": combined_categories
    }

    with open(output_ann_path, "w") as f:
        json.dump(combined_data, f, indent=4)

    print("\nVeri birleştirme tamamlandı.")

# --- Birleştirme İşlemini Başlat ---
paths_to_combine = [
    (annotations_path_1, source_dir_1),
    (annotations_path_2, source_dir_2)
]
combine_coco_datasets(paths_to_combine, combined_annotations_path, combined_dataset_dir)


# === 3. Analiz için Sınıf İsimleri ve Metadata ===
class_names = [
    "defect", "1- patlak", "2- igne_kirigi", "3- jut", "5- likra_kacigi", "6- yag_lekesi", "8- May cizgisi", "fsa"
]

unique_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
dataset_name = f"tekstil_combined_dataset_{unique_suffix}"

try:
    register_coco_instances(dataset_name, {}, combined_annotations_path, combined_dataset_dir)
except ValueError:
    pass

MetadataCatalog.get(dataset_name).thing_classes = class_names
metadata = MetadataCatalog.get(dataset_name)

# === 4. Model Yükleme ===
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)
print("\nModel loaded with Detectron2 DefaultPredictor.")


# === 5. Tahmin ve Karşılaştırma ===
gt_map = {}
filename_to_id = {}
all_image_paths = []

with open(combined_annotations_path, "r") as f:
    coco_ann = json.load(f)
    for img in coco_ann["images"]:
        filename = img["file_name"]
        img_id = img["id"]
        full_path = os.path.join(combined_dataset_dir, filename)
        filename_to_id[filename] = img_id
        all_image_paths.append(full_path)
    for ann in coco_ann["annotations"]:
        gt_map.setdefault(ann["image_id"], []).append(ann["category_id"])

y_true, y_pred = [], []
print(f"\nAnalyzing {len(all_image_paths)} images...")
for image_path in tqdm(all_image_paths):
    img = cv2.imread(image_path)
    if img is None:
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    pred_classes = instances.pred_classes.numpy().tolist()

    image_id = filename_to_id.get(os.path.basename(image_path))
    gt_classes = gt_map.get(image_id, [])

    for i in range(len(gt_classes)):
        y_true.append(gt_classes[i])
        if i < len(pred_classes):
            y_pred.append(pred_classes[i] + 1)
        else:
            y_pred.append(-1)

    for i in range(len(gt_classes), len(pred_classes)):
        y_true.append(-2)
        y_pred.append(pred_classes[i] + 1)

    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]
    save_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(save_path, out_img)

# --- 6. Genel Başarı Matrisi (TP/FP/FN/TN) Oluşturma ---

# Tüm defect sınıflarını "Hatalı" olarak, None ve Diğer sınıfları "Hatasız" olarak kabul etme
defect_class_ids = set(range(1, len(class_names) + 1))
y_true_binary = [1 if val in defect_class_ids else 0 for val in y_true]
y_pred_binary = [1 if val in defect_class_ids else 0 for val in y_pred]

# 2x2 karışıklık matrisini hesapla
cm = confusion_matrix(y_true_binary, y_pred_binary, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()
total_samples = tn + fp + fn + tp

# Matrisi istenen düzende görselleştirme
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_title('Genel Başarı Matrisi', fontsize=18, pad=20)
ax.set_xlabel('Tahmin Edilen', fontsize=14)
ax.set_ylabel('Gerçek', fontsize=14)

# Renk haritası için veri
heatmap_data = np.array([[tp, fp], [fn, tn]])
normalized_data = heatmap_data / np.max(heatmap_data) if np.max(heatmap_data) > 0 else np.zeros((2,2))

# Tek bir mavi renk tonu kullanma
cmap = plt.cm.get_cmap('Blues')
ax.imshow(normalized_data, cmap=cmap, interpolation='nearest', origin='upper')

# Metinleri hücrelere yazma
labels_text = [
    [f"TP\n{tp}\n({tp/total_samples*100:.2f}%)", f"FP\n{fp}\n({fp/total_samples*100:.2f}%)"],
    [f"FN\n{fn}\n({fn/total_samples*100:.2f}%)", f"TN\n{tn}\n({tn/total_samples*100:.2f}%)"]
]
for i in range(2):
    for j in range(2):
        color = 'white' if normalized_data[i, j] > 0.5 else 'black'
        ax.text(j, i, labels_text[i][j], ha='center', va='center', color=color, fontsize=16, weight='bold')

# Eksen etiketlerini ve işaretlerini ayarla
ax.set_xticks(np.arange(2))
ax.set_yticks(np.arange(2))
ax.set_xticklabels(["Hatalı", "Hatasız"])
ax.set_yticklabels(["Hatalı", "Hatasız"])

plt.tight_layout()
matrix_path = os.path.join(output_dir, "genel_basari_matrisi.png")
plt.savefig(matrix_path, bbox_inches='tight', pad_inches=0.5)
plt.close()

# Sınıflandırma raporunu (precision, recall, f1-score) kaydet
report = classification_report(
    y_true_binary, y_pred_binary, labels=[0, 1],
    target_names=["Hatasız", "Hatalı"],
    zero_division=0
)
report_path = os.path.join(output_dir, "binary_classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)

print(f"\nAnaliz tamamlandı. Genel Başarı Matrisi buraya kaydedildi: {output_dir}")

In [ ]:
import os
import shutil
from google.colab import drive

# Google Drive'ı bağlama (Colab için)
try:
    drive.mount('/content/drive')
except ImportError:
    pass

# === Dosya Yolları ===
# Taşınacak kaynak dosya (yeni konumu)
source_file = "/#your file path/ikinci.coco.json"

# Dosyanın eski konumu (hedef klasör)
destination_dir = "/#your file path/train"
destination_path = os.path.join(destination_dir, "ikinci.coco.json")

# === Dosyayı Taşıma ===
if os.path.exists(source_file):
    try:
        shutil.move(source_file, destination_path)
        print(f"'{source_file}' dosyası başarıyla '{destination_dir}' klasörüne geri taşındı.")
    except Exception as e:
        print(f"Dosyayı taşırken bir hata oluştu: {e}")
else:
    print(f"Hata: '{source_file}' dosyası belirtilen yolda bulunamadı.")

In [ ]:
import os
import shutil

# Google Drive'ı bağlama (Colab için)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

# === Dosya Yolları ===
# Taşınacak kaynak dosyalar
source_file_1 = "/#your file path/ikinci.coco.json"
source_file_2 = "/#your file path/train/ilk.coco.json"

# Dosyaların taşınacağı hedef klasör.
destination_dir = "/content/drive/MyDrive/jsonss"

# === 1. Hedef Klasörü Oluşturma ===
# Eğer 'jsonss' klasörü yoksa, oluştur.
os.makedirs(destination_dir, exist_ok=True)
print(f"Hedef klasör hazır: {destination_dir}\n")

# === 2. Dosyaları Taşıma ===
files_to_move = [source_file_1, source_file_2]

for file_path in files_to_move:
    # Dosya adını al
    file_name = os.path.basename(file_path)

    # Yeni dosya yolunu oluştur
    destination_path = os.path.join(destination_dir, file_name)

    if os.path.exists(file_path):
        try:
            # Dosyayı taşı
            shutil.move(file_path, destination_path)
            print(f"'{file_name}' dosyası başarıyla taşındı.")
        except Exception as e:
            print(f"'{file_name}' dosyasını taşırken bir hata oluştu: {e}")
    else:
        print(f"Hata: '{file_name}' dosyası kaynak yolda bulunamadı.")

print("\nTaşıma işlemi tamamlandı.")

In [ ]:
import os

ann_path = "/#your file path3/train/_annotations.coco.json"
print("Dosya var mı?", os.path.exists(ann_path))


In [ ]:
from detectron2 import _C   # detectron2 nin yol uyumsuzlugu çıkınca kullandığım kod

print(_C.__file__)


In [ ]:
# Gerekli kütüphaneleri içe aktarma
import torch
import detectron2
from detectron2.utils.logger import setup_logger
import os
import json
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from google.colab.patches import cv2_imshow
import cv2

# Google Drive'ı bağlama
from google.colab import drive
drive.mount('/content/drive')

# Logger'ı ayarlama
setup_logger()

# 1. Eğitilmiş Modeli ve Konfigürasyon Dosyalarını Yükleme
print("Eğitilmiş model dosyaları yükleniyor...")

# Model ve konfigürasyon dosyalarının yolları
model_dir = "/#your file path/model1"
model_path = os.path.join(model_dir, "model_final.pth")
config_path = os.path.join(model_dir, "config.yml")

# Konfigürasyon dosyasını yükle
cfg = get_cfg()
cfg.merge_from_file(config_path)

# Eğitilmiş modelin ağırlıklarını yükle
cfg.MODEL.WEIGHTS = model_path

# Düşük güvenilirlikteki tahminleri filtrelemek için eşik değerini düşürdük.
# Bu, modelin daha fazla nesne bulmasını sağlayacaktır.
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3  # Hassasiyet düşürüldü.
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Eğitilmiş model ile tahmin yapmak için bir öngörücü (predictor) oluştur
predictor = DefaultPredictor(cfg)

# 2. Resimleri Analiz Etme ve Görselleştirme
print("Foto-Arsivi/lab_data2_gray klasöründeki fotoğraflar analiz ediliyor...")

# Analiz edilecek resimlerin bulunduğu klasörün yeni yolu
test_image_dir = "/#your file path/lab_data2_gray"

# JPG ve PNG uzantılı tüm dosyaları listele
image_files = [f for f in os.listdir(test_image_dir) if f.lower().endswith(('.jpg', '.png'))]

if not image_files:
    print("Hata: Belirtilen klasörde hiç JPG veya PNG dosyası bulunamadı. Lütfen yolu ve dosya uzantısını kontrol edin.")
else:
    # Her bir resim dosyası için tahmin yap
    for filename in image_files:
        image_path = os.path.join(test_image_dir, filename)

        img = cv2.imread(image_path)

        if img is None:
            print(f"Hata: {filename} dosyası okunamadı. Atlama yapılıyor.")
            continue

        outputs = predictor(img)

        # Görselleştirme için Visualizer oluştur
        v = Visualizer(img[:, :, ::-1], scale=1.0)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

        print(f"Analiz edilen dosya: {filename}")
        cv2_imshow(out.get_image()[:, :, ::-1])

print("\nNesne tespiti ve görselleştirme tamamlandı.")


In [ ]:
import os
import json
import torch
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import Visualizer

from google.colab import drive

# 1. Google Drive'ı bağla
drive.mount('/content/drive')

# 2. Yolları ayarla
model_dir = "/#your file path/model3"
test_dir = "/#your file path/test"
test_json = os.path.join(test_dir, "_annotations.coco.json")
gray_dir = "/#your file path/lab_data2_gray"  # Örnek etiket yok klasör

output_dir = "/content/drive/MyDrive/Detectron2_Results"
os.makedirs(output_dir, exist_ok=True)

test_vis_dir = os.path.join(output_dir, "test_visualizations")
gray_vis_dir = os.path.join(output_dir, "gray_visualizations")
os.makedirs(test_vis_dir, exist_ok=True)
os.makedirs(gray_vis_dir, exist_ok=True)

# 3. Test datasını COCO olarak kayıt et
dataset_name = "tekstil_test_v2"
register_coco_instances(dataset_name, {}, test_json, test_dir)

# 4. COCO JSON'dan sınıf isimlerini çek ve metadata'ya ver
with open(test_json, "r") as f:
    coco_data = json.load(f)
thing_classes = [cat["name"] for cat in coco_data["categories"]]
MetadataCatalog.get(dataset_name).thing_classes = thing_classes
metadata = MetadataCatalog.get(dataset_name)

# 5. Detectron2 modeli yükle ve ayarla
cfg = get_cfg()
cfg.merge_from_file(os.path.join(model_dir, "config.yml"))
cfg.MODEL.WEIGHTS = os.path.join(model_dir, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2  # Hassasiyet artırıldı (eşik düşürüldü)
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)

# 6. Test görüntülerinde tahmin yap, görselleştir ve confusion matrix için gerçek ve tahminleri topla
dataset_dicts = DatasetCatalog.get(dataset_name)

y_true = []
y_pred = []

print("Test görüntüleri üzerinde tahmin ve confusion matrix hesaplanıyor...")
for data in dataset_dicts:
    img_path = data["file_name"]
    img = cv2.imread(img_path)
    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")

    # Gerçek sınıflar (etiketler)
    gt_classes = [ann["category_id"] for ann in data["annotations"]]
    # Tahmin edilen sınıflar
    pred_classes = instances.pred_classes.tolist()

    y_true.extend(gt_classes)
    y_pred.extend(pred_classes)

    # Görselleştir ve kaydet
    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]

    save_path = os.path.join(test_vis_dir, os.path.basename(img_path))
    cv2.imwrite(save_path, out_img)

print("Test görselleri tahminleri ve görselleri kaydedildi.")

# Confusion matrix çiz
labels = list(range(len(thing_classes)))
cm = confusion_matrix(y_true, y_pred, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=thing_classes)

plt.figure(figsize=(12, 8))
disp.plot(xticks_rotation=45, cmap="Blues", values_format="d")
plt.title("Confusion Matrix - Test Set")
plt.tight_layout()
cm_path = os.path.join(output_dir, "confusion_matrix.png")
plt.savefig(cm_path)
plt.show()
print(f"Confusion matrix görseli kaydedildi: {cm_path}")

# 7. Etiketsiz gray klasöründeki görüntüler için sadece tahmin yap ve görsel kaydet
print("\nEtiketsiz (gray) klasöründeki görüntüler tahmin ediliyor ve kaydediliyor...")

gray_images = [f for f in os.listdir(gray_dir) if f.lower().endswith(('.jpg', '.png'))]

for filename in gray_images:
    img_path = os.path.join(gray_dir, filename)
    img = cv2.imread(img_path)
    if img is None:
        print(f"Uyarı: {filename} okunamadı, atlanıyor.")
        continue

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")

    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(instances)
    out_img = out.get_image()[:, :, ::-1]

    save_path = os.path.join(gray_vis_dir, filename)
    cv2.imwrite(save_path, out_img)

print("Gray klasöründeki görseller işlenip kaydedildi.")
print("\nTüm işlem tamamlandı.")


In [ ]:
import json

# JSON dosya yolu
json_path = "/#your file path/test/_annotations.coco.json"

# JSON dosyasını aç ve içeriğini yükle
with open(json_path, "r") as f:
    coco_data = json.load(f)

# "categories" listesinden sınıf isimlerini çıkar
thing_classes = [cat["name"] for cat in coco_data["categories"]]

# Sonuçları yazdır
print("Sınıf isimleri (thing_classes):", thing_classes)


In [ ]:
# Gerekli kütüphaneleri içe aktarma
import cv2
import os
from google.colab import drive

# Google Drive'ı bağlama
drive.mount('/content/drive')

# 1. Klasör yollarını ve çıktı ayarlarını belirleme
# LÜTFEN BU YOLLARIN VE AYARLARIN DOĞRU OLDUĞUNDAN EMİN OLUN!

# Analiz edilecek resimlerin bulunduğu kaynak klasör
source_dir = "/#your file path/lab_data2_HD"

# İşlenmiş resimlerin kaydedileceği hedef klasör
# Bu klasör otomatik olarak oluşturulacaktır.
destination_dir = "/#your file path/lab_data2_gray"

# Resimlerin küçültüleceği hedef boyut (genişlik, yükseklik)
# Örnek: (640, 480) veya (1280, 720) gibi bir değer girebilirsiniz.
target_size = (640, 480)

# 2. Klasörlerin varlığını kontrol etme ve oluşturma

# Kaynak klasörün varlığını kontrol et
if not os.path.exists(source_dir):
    print(f"Hata: Kaynak klasör bulunamadı: {source_dir}")
    exit()

# Hedef klasörü oluştur
os.makedirs(destination_dir, exist_ok=True)
print(f"Hedef klasör oluşturuldu veya zaten mevcut: {destination_dir}\n")

# 3. Resimleri işleme ve kaydetme

print("Resimler işlenmeye başlıyor...")

# Kaynak klasördeki tüm dosyaları döngüye al
for filename in os.listdir(source_dir):
    # Dosyanın PNG formatında olup olmadığını kontrol et
    if filename.lower().endswith('.png'):
        source_path = os.path.join(source_dir, filename)

        # Resmi oku (OpenCV, resimleri varsayılan olarak BGR formatında okur)
        img = cv2.imread(source_path)

        if img is None:
            print(f"Hata: {filename} dosyası okunamadı. Atlama yapılıyor.")
            continue

        try:
            # Resim boyutunu küçültme
            resized_img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

            # Yeni dosya adı ve yolu oluşturma
            # .png uzantısını .jpg ile değiştir
            new_filename = os.path.splitext(filename)[0] + '.jpg'
            destination_path = os.path.join(destination_dir, new_filename)

            # Boyutu küçültülmüş renkli resmi JPG olarak kaydetme
            # Artık gri tonlamaya dönüştürme işlemi yapılmıyor.
            cv2.imwrite(destination_path, resized_img)

            print(f"'{filename}' başarıyla işlendi ve '{new_filename}' olarak kaydedildi.")

        except Exception as e:
            print(f"Hata: '{filename}' dosyası işlenirken bir sorun oluştu. Hata: {e}")

print("\nTüm resimler başarıyla işlendi ve kaydedildi.")
